# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

S. Belladitta  ->  S. Belladitta  |  ['S. Belladitta']
F. Walter  ->  F. Walter  |  ['F. Walter']
Z.-L. Xie  ->  Z.-L. Xie  |  ['Z.-L. Xie']
Y. Khusanova  ->  Y. Khusanova  |  ['Y. Khusanova']
D. Semenov  ->  D. Semenov  |  ['D. Semenov']
T. Henning  ->  T. Henning  |  ['T. Henning']
K. Paterson  ->  K. Paterson  |  ['K. Paterson']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
M. Flock  ->  M. Flock  |  ['M. Flock']
P. Mollière  ->  P. Mollière  |  ['P. Mollière']
Arxiv has 72 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2407.07236


extracting tarball to tmp_2407.07236...

 done.


E. Bañados  ->  E. Bañados  |  ['E. Bañados']
S. Belladitta  ->  S. Belladitta  |  ['S. Belladitta']
F. Walter  ->  F. Walter  |  ['F. Walter']
Z.-L. Xie  ->  Z.-L. Xie  |  ['Z.-L. Xie']
Y. Khusanova  ->  Y. Khusanova  |  ['Y. Khusanova']


...

Found 53 bibliographic references in tmp_2407.07649/aanda.bbl.
Retrieving document from  https://arxiv.org/e-print/2407.07678


extracting tarball to tmp_2407.07678...

 done.


P. Mollière  ->  P. Mollière  |  ['P. Mollière']


Found 129 bibliographic references in tmp_2407.07678/main.bbl.
syntax error in line 5: unbalanced braces


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.07236-b31b1b.svg)](https://arxiv.org/abs/2407.07236) | **A blazar in the epoch of reionization**  |
|| E. Banados, et al. -- incl., <mark>S. Belladitta</mark>, <mark>F. Walter</mark>, <mark>Z.-L. Xie</mark>, <mark>Y. Khusanova</mark> |
|*Appeared on*| *2024-07-11*|
|*Comments*| *Submitted*|
|**Abstract**|            Relativistic jets are thought to play a crucial role in the formation of massive galaxies and supermassive black holes. Here we report multi-wavelength and multi-epoch observations of the quasar VLASSJ0410-0139 at redshift z=7, powered by a 7e8 solar-mass black hole. Its radio variability, X-ray properties, and compact radio emission on parsec scales reveal that J0410-0139 is a blazar with a relativistic jet aligned with our line of sight. This blazar's existence implies that many more similar (unaligned) jetted sources must exist at z=7. One scenario is that we observe an intrinsically low-power radio jet, but we see it at high luminosity due to relativistic beaming effects. In this case, a large fraction (>80%) of the UV bright quasars must have a similar jet to match the number density expected from the UV quasar luminosity function. These jets can enhance the growth of supermassive black holes and substantially affect their host galaxies. However, the implications would be even more severe if the quasar belongs to the top 10% radio luminous quasars, as measured if the beaming enhancement is less than a factor of 10-15. In this scenario, there should be hundreds to thousands of radio-quiet quasars at z=7 with intrinsic properties similar to J0410-0139 -- in strong tension with the number density of bright quasars derived from their UV luminosity function. To reconcile these results, most black hole growth at z=7 must happen in an obscured phase, as some models predict. The existence of supermassive black holes in the epoch of reionization is facilitated by significant jet-enhanced or obscured super-Eddington accretion.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.07238-b31b1b.svg)](https://arxiv.org/abs/2407.07238) | **Chemistry in the GG Tau A Disk: Constraints from H2D+, N2H+, and DCO+ High Angular Resolution ALMA Observations**  |
|| P. Kashyap, et al. -- incl., <mark>D. Semenov</mark>, <mark>T. Henning</mark> |
|*Appeared on*| *2024-07-11*|
|*Comments*| *Resubmitted to ApJ. Thanks to the two anonymous referees for their constructive comments*|
|**Abstract**|            Resolved molecular line observations are essential for gaining insight into the physical and chemical structure of protoplanetary disks, particularly in cold, dense regions where planets form and acquire compositions. However, tracing these regions is challenging because most molecules freeze onto grain surfaces and are not observable in the gas phase. We investigated cold molecular chemistry in the triple stellar T Tauri disk GG Tau A, which harbours a massive gas and dust ring and an outer disk, using ALMA Band 7 observations. We present high angular resolution maps of N2H+ and DCO+ emission, with upper limits reported for H2D+, 13CS, and SO2. The radial intensity profile of N2H+ shows most emission near the ring outer edge, while DCO+ exhibits double peaks, one near the ring inner edge and the other in the outer disk. With complementary observations of lower-lying transitions, we constrained the molecular surface densities and rotation temperatures. We compared the derived quantities with model predictions across different cosmic ray ionization (CRI) rates, carbon-to-oxygen (C/O) ratios, and stellar UV fluxes. Cold molecular chemistry, affecting N2H+, DCO+, and H2D+ abundances, is most sensitive to CRI rates, while stellar UV flux and C/O ratios have minimal impact on these three ions. Our best model requires a low cosmic ray ionization rate of 1e-18 s-1. However, it fails to match the low temperatures derived from N2H+ and DCO+, 12 to 16 K, which are much lower than the CO freezing temperature.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.07257-b31b1b.svg)](https://arxiv.org/abs/2407.07257) | **Constraints on Relativistic Jets from the Fast X-ray Transient 210423 using Prompt Radio Follow-Up Observations**  |
|| D. Ibrahimzade, et al. -- incl., <mark>K. Paterson</mark> |
|*Appeared on*| *2024-07-11*|
|*Comments*| *14 pages, 6 figures*|
|**Abstract**|            Fast X-ray Transients (FXTs) are a new observational class of phenomena with no clear physical origin. This is at least partially a consequence of limited multi-wavelength follow up of this class of transients in real time. Here we present deep optical ($g-$ and $i-$ band) photometry with Keck, and prompt radio observations with the VLA of FXT 210423 obtained at ${\delta t \approx 14-36}$ days since the X-ray trigger. We use these multi-band observations, combined with publicly available data sets, to constrain the presence and physical properties of on-axis and off-axis relativistic jets such as those that can be launched by neutron-star mergers and tidal disruption events, which are among the proposed theoretical scenarios of FXTs. Considering a wide range of possible redshifts $z\le3.5$, circumstellar medium (CSM) density $n={10^{-6}-10^{-1}\,\rm{cm^{-3}}}$, isotropic-equivalent jet kinetic energy $E_{k,iso}={10^{48}-10^{55}\,\rm{erg}}$, we find that we can rule out wide jets with opening angle ${\theta_{j}=15^{\circ}}$ viewed within ${10^{\circ}}$ off-axis. For more collimated jets (${\theta_{j}=3^{\circ}}$) we can only rule out on-axis (${\theta_{j}=0^{\circ}}$) orientations. This study highlights the constraining power of prompt multi-wavelength observations of FXTs discovered in real time by current (e.g., Einstein Probe) and future facilities.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.07585-b31b1b.svg)](https://arxiv.org/abs/2407.07585) | **A photo-$z$ cautionary tale: Redshift confirmation of COSBO-7 at $z=2.625$**  |
|| S. Jin, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2024-07-11*|
|*Comments*| *Submitted to A&A Letter*|
|**Abstract**|            Photometric redshifts are widely used in studies of dusty star-forming galaxies (DSFGs), but catastrophic photo-$z$ failure can undermine all redshift-dependent results. Here we report the spectroscopic redshift confirmation of COSBO-7, a strongly lensed DSFG in the COSMOS-PRIMER field. Recently, using 10 bands of JWST NIRCam and MIRI imaging data on COSBO-7, Ling et al. (2024) reported a photometric redshift solution of $z\gtrsim7.0$, favored by four independent spectral energy distribution (SED) fitting codes, and therefore providing an appealing candidate of the most distant massive DSFG. This photo-$z$ solution was also supported by a single line detection in ALMA Band 3 consistent with CO(7-6) at $z=7.46$. However, our new ALMA observations robustly detect two lines in Band 6 identified as CO(7-6) and [CI](2-1) at $z_{\rm spec}=2.625$, and thus the Band 3 line as CO(3-2). The three robust line detections decidedly place COSBO-7 at $z=2.625$, refuting the photo-$z$ solution. We derive physical parameters by fitting NIR-to-mm photometry and lens modeling, revealing that COSBO-7 is a main-sequence galaxy. We examine possible reasons for this photo-$z$ failure and attribute it to (1) the likely underestimation of photometric uncertainty at 0.9$\mu$m, and (2) the lack of photometry at wavelengths beyond 20$\mu$m. Notably, we recover a bona-fide $z_{\rm phot}\sim 2.3$ by including the existing MIPS $24\mu$m photometry, demonstrating the critical importance of mid-infrared data for photo-$z$ robustness. This work highlights a common challenge in modeling SEDs of DSFGs, cautioning against the reliability of photometric redshifts, as well as pseudo-spectroscopic redshifts based on single line detection.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.07649-b31b1b.svg)](https://arxiv.org/abs/2407.07649) | **From traffic jams to roadblocks: The outer regions of TW Hya with ALMA Band 8**  |
|| S. Das, N. T. Kurtovic, <mark>M. Flock</mark> |
|*Appeared on*| *2024-07-11*|
|*Comments*| *Accepted for publication in A&A. The associated data can be found in: this https URL*|
|**Abstract**|            We present Atacama Large Millimeter Array (ALMA) observations of TW Hya at 0.65 mm with 0.5 arcsecond angular resolution, together with high angular resolution archival observations at 0.87 mm, 1.3 mm, 2.1 mm and 3.1 mm. We constrain the outer disk emission with both image-plane retrieval, and visibility-plane modeling with non-parametric and parametric fitting tools. Our results confirm emission in the outer disk regions of TW Hya (60 au < R < 110 au) at 0.65 mm, 0.87 mm and 1.3 mm. With image-plane retrieval, we resolve the new continuum gap and ring, namely D79 and B86, at 0.87 mm and 1.3 mm. With visibility-plane modeling, we also detect this substructure at 0.65 mm, and it is consistent in location with the outer ring proposed by Ilee et al. 2022 at the 1-sigma level. Furthermore, it has a high spectral index of 3.7, which may indicate dust grain sizes << 1 mm. It may be a dust trap or a traffic jam, that has a flux density of 60 mJy and a mass (1.59 earth masses) that accounts for up to 2% of the dust disk at 0.65 mm. In conclusion, we confirm the existence of a faint ring in the outer regions of TW Hya at multiple millimeter wavelengths. With visibility-plane modeling, we are able to set constrains that are 3 times better than the resolution of our Band 8 observations.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.07678-b31b1b.svg)](https://arxiv.org/abs/2407.07678) | **The ESO SupJup Survey II: The $^{12}$C/$^{13}$C ratios of three young brown dwarfs with CRIRES$^+$**  |
|| D. G. Picos, et al. -- incl., <mark>P. Mollière</mark> |
|*Appeared on*| *2024-07-11*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**|            Young brown dwarfs exhibit atmospheric characteristics similar to those of super-Jupiters, providing a unique opportunity to study planetary atmospheres. The ESO SupJup Survey, utilizing CRIRES$^+$ on the Very Large Telescope, aims to assess the role of $^{12}$C/$^{13}$C as a formation tracer. We present observations of three young brown dwarfs: 2MASS J12003792-7845082, TWA 28, and 2MASS J08561384-1342242, with the goal of constraining their chemical compositions, thermal profiles, surface gravities, spin rotations, and $^{12}$C/$^{13}$C. Atmospheric retrievals of CRIRES$^+$ K-band spectra were conducted using the radiative transfer code petitRADTRANS coupled with the Bayesian inference algorithm MultiNest, resulting in a detailed characterization of the atmospheres of these objects. We report the volume mixing ratios of main molecular and atomic species, including the novel detection of hydrogen fluoride (HF) in a brown dwarf's atmosphere, and determine $^{12}$C/$^{13}$C values of $81^{+28}_{-19}$ and $79^{+20}_{-14}$ in the atmospheres of TWA 28 and J0856, respectively, with strong significance ($>3\sigma$). Tentative evidence ($\sim 2\sigma$) of $^{13}$C in J1200 was found, with $^{12}$C/$^{13}$C = $114^{+69}_{-33}$, along with $^{18}$O detected at moderate significance in J0856 (3.3$\sigma$) and TWA 28 (2.1$\sigma$). The retrieved thermal profiles indicate hot atmospheres (2300-2600 K) with low surface gravities and slow spins, consistent with young objects. The consistent carbon isotope ratios among the three objects, showing no significant deviation from the local ISM, suggest a fragmentation-based formation mechanism similar to star formation. The tentative detection of $^{18}$O in two objects highlights the potential of high-resolution spectroscopy to probe additional isotope ratios, such as $^{16}$O/$^{18}$O, in the atmospheres of brown dwarfs and super-Jupiters.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.07336-b31b1b.svg)](https://arxiv.org/abs/2407.07336) | **Testing the cosmic distance duality relation using strong gravitational lensing time delays and Type Ia supernovae**  |
|| J.-Z. Qi, Y.-F. Jiang, W.-T. Hou, <mark>X. Zhang</mark> |
|*Appeared on*| *2024-07-11*|
|*Comments*| *11 pages, 3 figures*|
|**Abstract**|            We present a comprehensive test of the cosmic distance duality relation (DDR) using a combination of strong gravitational lensing (SGL) time delay measurements and Type Ia supernovae (SNe Ia) data. We investigate three different parameterizations of potential DDR violations. To bridge the gap between SGL and SNe Ia datasets, we implement an artificial neural network (ANN) approach to reconstruct the distance modulus of SNe Ia. Our analysis uniquely considers both scenarios where the absolute magnitude of SNe Ia ($M_B$) is treated as a free parameter and where it is fixed to a Cepheid-calibrated value. Using a sample of six SGL systems and the Pantheon+ SNe Ia dataset, we find no statistically significant evidence for DDR violations across all parameterizations. The consistency of our findings across different parameterizations not only reinforces confidence in the standard DDR but also demonstrates the robustness of our analytical approach.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.07323-b31b1b.svg)](https://arxiv.org/abs/2407.07323) | **The Age-Dependent Vertical Actions of Young Stars in the Galaxy**  |
|| D. N. Garzon, et al. -- incl., <mark>H.-W. Rix</mark> |
|*Appeared on*| *2024-07-11*|
|*Comments*| *10 pages, 5 figures. Draft version July 9, 2024. Typeset using LATEX default style in AASTeX631*|
|**Abstract**|            Stars in the Galactic disk are born on cold, nearly circular orbits with small vertical excursions. After their birth, their orbits evolve, driven by small- or large-scale perturbations in the Galactic disk's gravitational potential. Here, we study the vertical motions of young stars over their first few orbital periods, using a sample of OBA stars from \textit{Gaia} E/DR3, which includes radial velocities and ages $\tau$ from LAMOST. We constructed a parametric model for the time evolution of the stellar orbits' mean vertical actions $J_z$ as a function of Galactocentric radius, $R_{\mathrm{GC}}$. Accounting for data uncertainties, we use Markov Chain Monte Carlo (MCMC) analysis in annuli of Galactocentric radius to constrain the model parameters. Our best-fit model shows a remarkably linear increase of vertical actions with age across all Galactocentric radii examined. Orbital \textit{heating} by random scattering could offer a straightforward interpretation for this trend. However, various other dynamical aspects of the Galactic disk, such as stars being born in a warped disk, might offer alternative explanations that could be tested in the future.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2407.07236/figures/f2a.png', 'tmp_2407.07236/figures/f2b.png', 'tmp_2407.07236/figures/f2c.png', 'tmp_2407.07236/figures/f1.png', 'tmp_2407.07236/figures/f3.png']
copying  tmp_2407.07236/figures/f2a.png to _build/html/
copying  tmp_2407.07236/figures/f2b.png to _build/html/
copying  tmp_2407.07236/figures/f2c.png to _build/html/
copying  tmp_2407.07236/figures/f1.png to _build/html/
copying  tmp_2407.07236/figures/f3.png to _build/html/
exported in  _build/html/2407.07236.md
    + _build/html/tmp_2407.07236/figures/f2a.png
    + _build/html/tmp_2407.07236/figures/f2b.png
    + _build/html/tmp_2407.07236/figures/f2c.png
    + _build/html/tmp_2407.07236/figures/f1.png
    + _build/html/tmp_2407.07236/figures/f3.png
found figures ['tmp_2407.07238/./Integrated_intensity_radial_14th.png', 'tmp_2407.07238/./kep-n2h+43.png', 'tmp_2407.07238/./kep-dco+54.png', 'tmp_2407.07238/./int_spetra.png']
copying  tmp_2407.07238/./Integrated_intensity_radial_14th.png to _build/html/
c

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\fhi}{\overline{x}_{\mathrm{H}\ensuremath{\textsc{i}}}}$
$\newcommand{\hi}{H\ensuremath{ \textsc{i}}}$
$\newcommand{\oi}{O\ensuremath{ \textsc{i}}}$
$\newcommand{\mgii}{Mg\ensuremath{ \textsc{ii}}}$
$\newcommand{\cii}{[C\ensuremath{ \textsc{ii}}]}$
$\newcommand{\ciib}{C\ensuremath{ \textsc{ii}}}$
$\newcommand{\ciii}{C\ensuremath{ \textsc{iii}}]}$
$\newcommand{\civ}{C\ensuremath{ \textsc{iv}}}$
$\newcommand{\feii}{Fe\ensuremath{ \textsc{ii}}}$
$\newcommand{\siii}{Si\ensuremath{ \textsc{ii}}}$
$\newcommand{\farcs}{\mbox{.\!\!^{\prime\prime}}}$
$\newcommand{\kms}{{\rm km s}\ensuremath{^{-1}}}$
$\newcommand{\gt}{\ensuremath{>}}$
$\newcommand{\lt}{\ensuremath{<}}$
$\newcommand{\ap}{\ensuremath{\sim}}$
$\newcommand{\Jntt}{\ensuremath{J_{\rm SOFI}}}$
$\newcommand{\Hntt}{\ensuremath{H_{\rm SOFI}}}$
$\newcommand{\Kntt}{\ensuremath{Ks_{\rm SOFI}}}$
$\newcommand{\blazar}{J0410--0139}$
$\newcommand$
$\newcommand$
$\newcommand{\aap}{Astron. Astrophys.}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\pasa}{Public. of the Astron. Soc. of Australia}$
$\newcommand{\thefootnote}{\fnsymbol{footnote}}$
$\newcommand{\thefigure}{E\arabic{figure}}$
$\newcommand{\thetable}{E\arabic{table}}$</div>



<div id="title">

# A blazar in the epoch of reionization

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.07236-b31b1b.svg)](https://arxiv.org/abs/2407.07236)<mark>Appeared on: 2024-07-11</mark> -  _Submitted_

</div>
<div id="authors">

<mark>E. Bañados</mark>, et al. -- incl., <mark>S. Belladitta</mark>, <mark>F. Walter</mark>, <mark>Z.-L. Xie</mark>, <mark>Y. Khusanova</mark>

</div>
<div id="abstract">

**Abstract:** **Relativistic jets are thought to play a crucial role in the formation of massive galaxies and supermassive black holesblandford2019**.Here we report multi-wavelength and multi-epoch observations of the quasar VLASS J041009.05$-$013919.88 (hereafter J0410--0139) at redshift $z=7.0$,  powered by a $7\times10^8 M_\odot$ black hole.Its radio variability,  X-ray properties,  and compact radio emission on parsec scales reveal that J0410--0139 is a blazar with a relativistic jet aligned with our line of sight.  This blazar's existence implies that many more similar (unaligned) jetted sources must exist at $z \sim 7$.One scenario is that we observe an intrinsically low-power radio jet,  but we see it at high luminosity due to relativistic beaming effects.  In this case,  a large fraction ($>80\%$) of the UV bright quasars must have a similar jet to match the number density expected from the UV quasar luminosity function\cite{matsuoka2023.  These jets can enhance the growth of supermassive black holesjolley2008 and substantially affect their host galaxiesventuri2021.However, the implications would be even more severe if the quasar belongs to the top 10\% radio luminous quasarsbanados2015a, liu2021, as measured if the beaming enhancement is less than a factor of $10-15$.In this scenario, there should be hundreds to thousands of radio-quiet quasars at $z \sim 7$ with intrinsic properties similar to \blazarvolonteri2011, ghisellini2013---in strong tension with the number density of bright quasars derived from their UV luminosity function. To reconcile these results, most black hole growth at $z \sim 7$ must happen in an obscured phase, as some models predictjohnson2022, satyavolu2023.The existence of supermassive black holes in the epoch of reionization is facilitated by significant jet-enhanced or obscured super-Eddington accretion.}

</div>

<div id="div_fig1">

<img src="tmp_2407.07236/figures/f2a.png" alt="Fig2.1" width="33%"/><img src="tmp_2407.07236/figures/f2b.png" alt="Fig2.2" width="33%"/><img src="tmp_2407.07236/figures/f2c.png" alt="Fig2.3" width="33%"/>

**Figure 2. -** \small**Spectral energy distribution (SED) and radio lightcurve of $\blazar$**. _**(a)**_: Rest-frame SED from radio to X-rays. Only the VLA 2021 simultaneous observations are shown in this panel. The X-ray region considers $2\sigma$ uncertainties in the X-ray photon index and flux densities. The green and orange SEDs are from low redshift blazarsmassaro2015 with similar radio and X-ray luminosities (5BZQJ1232--0224 at $z=1$ and 5BZQJ1443+6332 at $z=1.4$). The gray shaded regions are the 1 and 2$\sigma$ expected X-ray luminosity for a radio-quiet quasar with the same $L_{2500}$ as $\blazar$lusso2016.
_**(b)**: Radio SED in linear scale._ The data points are color-coded by year of observation for visualization purposes; the measurements with their exact dates are listed in Table \ref{tab:obs}.  The quasar is highly variable at all radio frequencies with multi-epoch observations (see also Fig. \ref{fig:discovery}). The diamonds (squares) are VLA 2021 (2022) observations taken in the same observing block, yielding a quasi-simultaneous view of the SED  (within 15 minutes in the rest frame).
 The dashed lines are power-law extrapolations from the low and high-frequency simultaneous measurements.
_**(c)**: 1.4 and 3.0  GHz radio lightcurve._ All data points are normalized to the first flux density at hand for each frequency. For the 1.4 GHz lightcurve we considered all data measurements between 1.37 and 1.5 GHz, and they are corrected to 1.4 GHz using $\alpha=0.85$ if observed before 2022 and $\alpha=0.21$ if observed after 2022 (see panel b).
 (*fig:sed*)

</div>
<div id="div_fig2">

<img src="tmp_2407.07236/figures/f1.png" alt="Fig1" width="100%"/>

**Figure 1. -** \small**Photometry and spectra of the quasar $\blazar$ at $z=7.0$.**
The grayscale postage stamps are optical and near-infrared images, while the radio images are shown with different colormaps (3 GHz in orange and 1.4 GHz in light blue).  The 1995 NVSS 1.4 and the 2022 VLA 3 GHz images were taken in the most compact VLA configuration and, therefore, have the lowest resolution (see Table \ref{tab:obs}). The 2021 VLA 1.4 GHz image is shown with the same size as the lower resolution images to demonstrate that there is only one radio source in the field. There is strong evidence for radio variability.
 The quasar was first spectroscopically identified on 7 November 2020 with a 30 min exposure with the FORS2 spectrograph at the VLT. The second-to-bottom panel compares the VLT/FORS2, Keck/NIRES, Magellan/FIRE, and LBT/LUCI spectra.
 All spectra of $\blazar$ in the rest-frame UV are consistent, with no evidence of variability in the broad line region or accretion disk.
 The bottom panel shows the combined spectrum (and the $1\sigma$ error in green). The inset shows a Gaussian fit to the $\mgii$ line, from which we derive a black hole mass of $6.9^{+0.5}_{-0.4}\times 10^8 M_\odot$ and an Eddington ratio of $L_{\rm bol}/L_{\rm Edd}=1.22\pm 0.08$.
 (*fig:discovery*)

</div>
<div id="div_fig3">

<img src="tmp_2407.07236/figures/f3.png" alt="Fig3" width="100%"/>

**Figure 3. -** \small**Very Long Baseline Array (VLBA) 1.5 GHz high angular resolution image (left), _Chandra**/ACIS 0.5--7.0 keV (center), and \textit{XMM_/EPIC 0.3--4.5 keV image (right) of $\blazar$}. _Left:_ The VLBA beam size is $14.3 \times 5.9$ mas ($ 75 \times 37$ pc).
The source is marginally resolved with a size $< 3$ mas ($<16$ pc).
_Center and Right:_$\blazar$ is unresolved in the _XMM_ and _Chandra_ images and is the strongest X-ray source in the field. VLBA and X-ray images reveal a single compact dominant source, supporting the blazar interpretation.
 (*fig:vlba_xmm*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.07236"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\nnhp}{N_2H^+ (4-3)}$
$\newcommand$
$\newcommand{\hhdp}{o-H_2D^+ (1(1,0)-1(1,1))}$
$\newcommand{\CS}{^{13}CS (8-7)}$
$\newcommand{\soo}{SO_2 19(4,16)-19(3,17)}$
$\newcommand{\textbf{}}[1]{#1}$
$\usepackage[version=4]{mhchem}$
$\usepackage{multirow}$
$\usepackage{graphicx}$
$\usepackage{color}$
$\begin{document}$
$\title{\large{Chemistry in the GG Tau A Disk: Constraints from H_2D^+, N_2H^+, and DCO^+ High Angular Resolution ALMA Observations}}$
$\correspondingauthor{Liton Majumdar}$
$\email{liton@niser.ac.in, dr.liton.majumdar@gmail.com}$
$\author[0009-0008-2350-5876]{Parashmoni Kashyap}$
$\author[0000-0001-7031-8039]{Liton Majumdar}$
$\affiliation{School of Earth and Planetary Sciences, National Institute of Science Education and Research, Jatni 752050, Odisha, India}$
$\affiliation{Homi Bhabha National Institute, Training School Complex, Anushaktinagar, Mumbai 400094, India}$
$\author{Anne Dutrey}$
$\author[0000-0003-3773-1870]{St{é}phane Guilloteau}$
$\affiliation{Laboratoire d'Astrophysique de Bordeaux, Université de Bordeaux, CNRS, B18N, Allée Geoffroy Saint-Hilaire, F-33615 Pessac, France}$
$\author[0000-0001-6124-5974]{Karen Willacy}$
$\affiliation{Jet Propulsion Laboratory, California Institute of Technology, 4800 Oak Grove Dr. Pasadena, CA, 91109, USA}$
$\author{Edwige Chapillon}$
$\affiliation{Institut de Radioastronomie Millimétrique (IRAM), 300 rue de la Piscine, F-38406 Saint-Martin d’Héres, France}$
$\affiliation{Laboratoire d'Astrophysique de Bordeaux, Université de Bordeaux, CNRS, B18N, Allée Geoffroy Saint-Hilaire, F-33615 Pessac, France}$
$\author[0000-0003-1534-5186]{Richard Teague}$
$\affiliation{ Department of Earth, Atmospheric, and Planetary Sciences, Massachusetts Institute of Technology, Cambridge, MA 02139, USA}$
$\author[0000-0002-3913-7114]{Dmitry Semenov}$
$\affiliation{Max Planck Institute for Astronomy, Königstuhl 17, D-69117 Heidelberg, Germany}$
$\affiliation{Department of Chemistry, Ludwig Maximilian University, Butenandtstr. 5–13, D-81377 Munich, Germany}$
$\author[0000-0002-1493-300X]{Thomas Henning}$
$\affiliation{Max Planck Institute for Astronomy, Königstuhl 17, D-69117 Heidelberg, Germany}$
$\author[0000-0001-8292-1943]{Neal Turner}$
$\author[0000-0002-6858-5063]{Raghvendra Sahai}$
$\affiliation{Jet Propulsion Laboratory, California Institute of Technology, 4800 Oak Grove Dr. Pasadena, CA, 91109, USA}$
$\author[0000-0001-7157-6275]{Ágnes Kóspál}$
$\affiliation{Konkoly Observatory, HUN-REN Research Centre for Astronomy and Earth Sciences, CSFK, MTA Centre of Excellence, Konkoly-Thege Miklós út 15-17, 1121 Budapest, Hungary}$
$\affiliation{ELTE Eötvös Loránd University, Institute of Physics, Pázmány Péter sétány 1/A, 1117 Budapest, Hungary}$
$\affiliation{Max Planck Institute for Astronomy, Königstuhl 17, D-69117 Heidelberg, Germany}$
$\author[0000-0003-1805-3920]{Audrey Coutens}$
$\affiliation{Institut de Recherche en Astrophysique et Planétologie (IRAP), Université de Toulouse, UT3-PS, CNRS, CNES, 9 av. du Colonel Roche, 31028 Toulouse Cedex 4, France}$
$\author[0009-0006-3497-397X]{V. Piétu}$
$\affiliation{Laboratoire d'Astrophysique de Bordeaux, Université de Bordeaux, CNRS, B18N, Allée Geoffroy Saint-Hilaire, F-33615 Pessac}$
$\author[0000-0002-6636-4304]{Pierre Gratier}$
$\affiliation{Laboratoire d'Astrophysique de Bordeaux, Université de Bordeaux, CNRS, B18N, Allée Geoffroy Saint-Hilaire, F-33615 Pessac}$
$\author[0000-0003-0522-5789]{Maxime Ruaud}$
$\affiliation{NASA Ames Research Center, Moffett Field, CA 94035, USA}$
$\affiliation{Carl Sagan Center, SETI Institute, Mountain View, CA 94035, USA}$
$\author[0000-0002-4372-5509]{N. T. Phuong}$
$\affiliation{Department of Astrophysics, Vietnam National Space Center, Vietnam Academy of Science and Techonology, 18 Hoang Quoc Viet, Cau Giay, Hanoi, Vietnam}$
$\author[0009-0009-9618-4927]{E. Di Folco}$
$\affiliation{Laboratoire d'Astrophysique de Bordeaux, Université de Bordeaux, CNRS, B18N, Allée Geoffroy Saint-Hilaire, F-33615 Pessac}$
$\author[0000-0002-3024-5864]{Chin-Fei Lee}$
$\author[0000-0002-0675-276X]{Y.-W. Tang}$
$\affiliation{Academia Sinica Institute of Astronomy and Astrophysics, PO Box 23-141, Taipei 106, Taiwan}$
$\begin{abstract}$
$    Resolved molecular line observations are essential for gaining insight into the physical and chemical structure of protoplanetary disks, particularly in cold, dense regions where planets form and acquire compositions. However, tracing these regions is challenging because most molecules freeze onto grain surfaces and are not observable in the gas phase. We investigated cold molecular chemistry in the triple stellar T Tauri disk GG Tau A, which harbours a massive gas and dust ring and an outer disk, using ALMA Band 7 observations. We present high angular resolution maps of \ce{N2H+} and \ce{DCO+} emission, with upper limits reported for \ce{H2D+}, ^{13}CS, and \ce{SO2}. The radial intensity profile of \ce{N2H+} shows most emission near the ring outer edge, while \ce{DCO+} exhibits double peaks, one near the ring inner edge and the other in the outer disk. With complementary observations of lower-lying transitions, we constrained the molecular surface densities and rotation temperatures. We compared the derived quantities with model predictions across different cosmic ray ionization (CRI) rates, carbon-to-oxygen (C/O) ratios, and stellar UV fluxes. \textbf{Cold molecular chemistry, affecting \ce{N2H+}, \ce{DCO+}, and \ce{H2D+} abundances, is most sensitive to CRI rates, while stellar UV flux and C/O ratios have minimal impact on these three ions. Our best model requires a low cosmic ray ionization rate of 10^{-18} s^{-1}. However, it fails to match the low temperatures derived from \ce{N2H+} and \ce{DCO+}, 12 to 16 K, which are much lower than the CO freezing temperature.}$
$\end{abstract}$
$\keywords{Protoplanetary disks (1300); Planet formation (1241); Astrochemistry (75)}$
$\section{Introduction} \label{sec:intro}$
$A protoplanetary disk is a crucial stage in the evolution of cosmic matter, where the interplay of inherited material from a parent cloud and the in-situ physics and chemistry establishes the conditions for planet formation. Protoplanetary disks are characterized by complex chemical stratification owing to radial and vertical variations in density, temperature, ionization, and dissociative radiation  ([Aikawa and Herbst 2002]()) . The uppermost layers usually allow the formation of simple atoms, ions, and photo-stable radicals, while the intermediate warm molecular layer produces most of the gas phase molecules traced by the  Atacama Large Millimeter/submillimeter Array (ALMA) observations. There is minimal penetration of stellar radiation in \textbf{the vertically innermost regions near the midplane}, making them very cold and poorly ionized. In this dense region, most of the molecules freeze onto grain surfaces. Hence, midplane regions are very difficult to probe. N_2H^+, DCO^+, and H_2D^+ are among the very few molecules which remain in the gas phase in the midplane region  ([Ceccarelli, et. al 2004](), [Öberg, et. al 2011]()) .\par$
$N_2H^+ is expected to be abundant in regions where \ce{CO} is depleted. This anti-correlation is observed in both dense cores \citep[e.g.,][]{Caselli1999, Bergin2002} as well as in disks  ([Walsh, et. al 2012]()) . \ce{N2H+} is produced through the protonation of \ce{N2} and rapidly destroyed by CO in its presence to form \ce{HCO+}.$
$Hence, N_2H^+ can persist in the gas phase \textbf{at temperatures a few Kelvin} below the CO snowline  ([Öberg and Fraser 2005]()) . N_2H^+ has been observed in multiple disks \citep[e.g.,][]{Dutrey2007, Qi2013, Phuong2021, Anderson2022}, but is often characterized by comparatively weak emission.\par$
$An enhanced abundance of H_2D^+ is predicted by chemical models at low temperatures (T<20 K), with the major formation pathway being protonation of \ce{HD}  ([Roberts and Millar 2000]()) . Hence, H_2D^+ can only be observed in \textbf{high-density regions with low \ce{CO} and \ce{N2} content} where \ce{H3+} has not already been destroyed. H_2D^+ has already been detected in prestellar cores \citep[e.g.,][]{Caselli2008}, but there is no clear detection in any of the protoplanetary disks \citep[e.g.,][]{Ceccarelli2004,Thi2004,Qi2008, Chapillon2011}. \par$
$DCO^+ serves as an effective indicator of low-temperature CO freezeout regions within the disk and is predominantly formed through the interaction of \ce{CO} with H_2D^+  (Wootten1987) . \textbf{On the one hand, the formation of one of the parent molecules, H_2D^+, mainly occurs through HD protonation and is hindered by \ce{H3+} destruction in the presence of CO. On the other hand, the presence of CO is essential for DCO^+ formation.} Consequently, DCO^+ emission is expected to peak in a narrow region around the \ce{CO} snowline where both the parent molecules are in the gas phase in small amounts  ([Aikawa and Herbst 2002](), [ and Willacy 2007](), [Mathews, Klaassen and Juhász 2013]()) .\par$
$In summary, N_2H^+, DCO^+, and H_2D^+ are excellent tracers of low-temperature, high-density regions of the disk, specifically the midplane region in outer disks. Simultaneous observations of these molecules allow us to locate the CO snowline. \textbf{Snowlines are thought to play a significant role in planet formation since ice coating of grains enhances the solid mass, potentially promotes coagulation of grains to larger particles, and/or requires a higher collision velocity for destruction, effects which are particularly prominent just outside the snowline  ([Okuzumi, et. al 2012]()) .} Additionally, such observations new insights into ionization fractions within the relevant disk areas, offering constraints that can impact magneto-rotational instability. In instances of excessively low ionization fractions, "dead zones"  ([ and Gammie 1996]())  form, which are conducive to efficient grain growth and subsequent planet formation \citep[e.g.,][]{Oberg2011}. \par$
$This work reports spatially resolved emission from \nnhp and \dcop and upper limits for \hhdp, \CS, and \soo in the disk around a triple stellar T Tauri system, GG Tau A, utilizing the ALMA interferometer. \textbf{This represents the most sensitive observation of these molecules in GG Tau A to date.} The system (age 1-5 Myr) is located at a distance of 150 pc \citep[\textbf{using the GAIA parallax measurements of the GG Tau Ba star of this hierarchical quintuple system, because those of GG Tau A are contaminated by its triple nature,}][]{Gaia2016,Gaia2018} from us in the Taurus-Auriga star-forming region. \textbf{This circumtertiary disk is characterized by a dense ring of gas and dust between 193 and 285 au (from here on, we simply use the term ring to denote this region) and an outer disk extended up to 800 au in ^{13}CO observations  ([Guilloteau, Dutrey and Simon 1999](), [Dutrey and Guilloteau 2014]()) . The majority of the total disk mass (0.15 M_{\odot}) is concentrated in the ring region (0.13 M_{\odot})  ([Guilloteau, Dutrey and Simon 1999](), [Andrews, Chandler and Isella 2014]()) }. [Dutrey and Guilloteau (2014)]() reported the presence of a hot spot located at the outer edge of the dense ring where a partial CO gap is observed  ([Tang, Dutrey and Guilloteau 2016]()) , suggesting the presence of an embedded planet. This hypothesis is reinforced by the detection of a CO spiral emanating from the hot spot  ([Phuong and Dutrey 2020]()) .$
$This disk is very cold, with a midplane temperature of \sim 14 K  ([Dutrey and Guilloteau 2014]())  and an atmospheric temperature of \sim 30 K  ([Guilloteau, Dutrey and Simon 1999](), [Phuong, Dutrey and Diep 2020]()) , at a radial distance of 214 au from the center. The first-ever detection of \ce{H2S} in the outer disk of GG Tau A by [Phuong, Chapillon and Majumdar (2018)]() further confirms how dense and massive this system is. Given its substantial mass, high density (which should ease the detection of weak molecular lines), and low temperature, GG Tau A emerges as an ideal candidate for unravelling the intricacies of cold molecular chemistry within a protoplanetary disk. \par$
$We present our observational details in Section \ref{sec:obs} of this paper. We explain the schemes we undertake to reduce the data in Section \ref{subsec: data_red} and the observational results in Section \ref{subsec:obs_res}, including integrated intensity maps, radial profiles, and Keplerian deprojected spectra. We also constrain the surface density of the detected molecules and the upper limits of surface density for the undetected molecules, as explained in Section \ref{sec:constr_disk_phy_params}. We perform a chemical analysis to understand our observations better using our astrochemical model, detailed in Section \ref{sec:models}. We discuss our results in Section \ref{sec:discussions} and summarize the key understandings of our study in Section \ref{sec:conclusion}.$
$\section{Observations}$
$\label{sec:obs}$
$\begin{deluxetable*}{ccccccc}[!ht]$
$\tablecaption{Spectral Setup of the ALMA Band 7 Observations of GG Tau A}$
$\label{tab:spectral_setup}$
$\tablehead{$
$  \colhead{\multirow{2}{*}{Spectral Window}} &$
$  \colhead{\multirow{2}{*}{Central Frequency (GHz)}} &$
$  \multicolumn{2}{c}{Bandwidth} &$
$  \multicolumn{2}{c}{Resolution} &$
$  \colhead{\multirow{2}{*}{No. of Channels}} \   & & \colhead{Frequency (MHz)} & \colhead{Velocity (km/s)} & \colhead{Frequency (MHz)} & \colhead{Velocity (km/s)} &$
$}$
$\startdata$
$25 & 358.000000 & 1875.00 & 1570.2 & 1.129 & 0.945 & 3840 \ 27 & 359.770685 & 117.19 & 97.7 & 0.141 & 0.118 & 1920 \ 29 & 360.169780 & 117.19 & 97.5 & 0.141 & 0.117 & 1920 \ 31 & 369.908554 & 117.19 & 95.0 & 0.141 & 0.114 & 1920 \ 33 & 372.421385 & 117.19 & 94.3 & 0.122 & 0.098 & 1920 \ 35 & 372.672493 & 58.59 & 47.1 & 0.141 & 0.114 & 960$
$\enddata$
$\end{deluxetable*}$
$The GG Tau A system (ICRS 04:32:30.3460, 17:31:40.642) was observed as part of the ALMA project \#2021.1.00342.S (PI: Liton Majumdar) targeting to investigate the presence of Band 7 transitions, \hhdp, \nnhp, \dcop  \CS and \soo. The observations were made over two executions using 47 antennas with baselines ranging from 14 m to 456 m for a total on-source integration time of 2.7 hours. The spectral setup details can be found in Table \ref{tab:spectral_setup}.$
$\subsection{Data Reduction} \label{subsec: data_red}$
$Calibration procedures involved bandpass and flux calibration, with the quasar source J0423-0120 utilized for this purpose. Phase calibration was conducted using the sources J0510+1800 and J0440+1437. The observed data were initially calibrated with CASA, the Common Astronomy Software Applications package (version 6.4.1.12) pipeline. Subsequently, five rounds of continuum self-phase calibration were performed for each execution, selecting line-free channels carefully. The self-calibration solutions were then applied to the entire dataset. To determine the disk center, we fitted the continuum structure with an ellipse, \textbf{and both executions were recentered to the disk center using the \texttt{fixvis} task in CASA.} The data with the identical spectral setups from both executions were then combined using the \texttt{concat} task in CASA. We \textbf{performed continuum subtraction} on the self-calibrated recentered data by fitting a first-order polynomial to the continuum channels using \texttt{uvcontsub} task in CASA. We constructed the images from our visibility dataset in  \texttt{tclean} task in CASA using \texttt{hogbom} deconvolving algorithm with \texttt{briggs} weighting (robust=1.0). This yielded an angular resolution of 0.93" \times 0.77"$
$at \textbf{Position Angle (PA)} 68.76^\circ, and a typical sensitivity of 14.6 mJy/beam for N_2H^+ (channel maps are presented in Figure \ref{fig:ch_maps_n2hp}). For DCO^+, we achieved an angular resolution of 0.91" \times 0.73"$
$at PA 99.20^\circ, with a sensitivity of 5.6 mJy/beam (channel maps are presented in Figure \ref{fig:ch_maps_dcop}). For H_2D^+, the noise \textbf{(16.7 mJy/beam)} is higher as the atmospheric transmission is degraded at this high frequency.$
$For the weaker lines, including H_2D^+, we have used \texttt{multiscale} deconvolver with \texttt{scales = [0,5,15,25]} \times pixels where each pixel corresponds to one-seventh of the beam size. We adopted the \texttt{natural} weighting scheme to enhance the signal-to-noise ratio for the weak lines. The masks used during cleaning were generated with the \texttt{keplerian\_mask.py} tool \footnote{\url{https://github.com/richteague/keplerian\_mask}} with the geometric parameters M_{*} =1.37 M_{\odot}, i = 37^{\circ}, and PA = 278^{\circ} confined within \textbf{inner and outer radii of 1" and 3.5"} respectively (values rescaled from [Guilloteau, Dutrey and Simon (1999)]() for a distance of 150 pc). Additionally, a continuum image was generated for a spectral window with a central frequency of 358 GHz, utilizing the line-free channels. The deconvolution algorithm employed was \texttt{hogbom}, and the process utilized the \texttt{briggs} weighting scheme with the robust parameter set to 0.$
$\subsection{Observational Results} \label{subsec:obs_res}$
$The integrated intensity maps were generated by collapsing the image cubes along the velocity axis while applying the Keplerian masks used for cleaning. We did not use any intensity threshold criteria to decide the inclusion of pixels while generating the moment map. For this purpose, we employed \texttt{bettermoments} python package  ([Teague and Foreman-Mackey 2018]()) . Figure \ref{fig:mo_maps} displays integrated intensity maps for the targeted molecules, where the elliptical contours indicate the location of the dust continuum ring region (193 au - 285 au) of the disk GG Tau. It also displays the continuum image at the central frequency of 358 GHz in the bottom right corner. Clear detections of \nnhp and \dcop are observed, while \hhdp  \CS and \soo show no detection.\par$
$\begin{figure*}[ht!]$
$    \centering$
$    \includegraphics[width=\textwidth]{M0_maps_22.pdf}$
$    \caption{Integrated intensity maps of the targeted lines \nnhp, \dcop and \hhdp. The overplotted elliptical ring contours in white are at 193 au (1.28") and 285 au (1.9") to indicate the continuum ring region where 90\% of the circumtertiary emission arises  ([Guilloteau, Dutrey and Simon 1999]()) . Both N_2H^+ and DCO^+ emissions are extended beyond the ring region. As far as H_2D^+ is concerned, there is no significant emission from the ring region. The fourth one is the continuum image at 358 GHz.$
$    }$
$    \label{fig:mo_maps}$
$\end{figure*}$
$We generated radial distribution of velocity integrated brightness across the disk from the zeroth moment maps with the help of \texttt{radial\_profile} function in \texttt{GoFish} python package  (GoFish)  by dividing the disk radial range into a series of annular rings \textbf{with a width 1/4th of the beam major axis} and averaging the emission for each ring. The radial profiles for \nnhp and \dcop are shown in Figure \ref{fig:radial_intensity}. The two vertical lines indicate the ring region (193 au - 285 au) where 90\% of the circumstellar continuum emission resides  ([Guilloteau, Dutrey and Simon 1999]()) . \textbf{We have also shown the radial profiles for all the transitions generated without keplerian masking in Figure \ref{fig:rad_profile_wihtout_kep}. \ce{DCO+} shows a slightly lower emission in the ring region in this case.}\par$
$\begin{figure*}[!ht]$
$    \centering$
$    \includegraphics[width=\textwidth]{Integrated_intensity_radial_14th.pdf}$
$    \caption{Azimuthally averaged velocity integrated radial profiles of the detected lines generated from the integrated intensity maps shown in Figure \ref{fig:mo_maps}. The shaded region in the above figure corresponds to 1\sigma. The dashed vertical lines are to designate the location of the ring around GG Tau A, spanning from 193 au to 285 au. The grey horizontal line at the top right corner represents the beam major axis.}$
$    \label{fig:radial_intensity}$
$\end{figure*}$
$We employed the shifting and stacking technique using the \texttt{integrated\_spectrum} function in the \texttt{GoFish} package  (GoFish)  to generate a disk-integrated spectrum. \textbf{Velocities for each pixel} were shifted to the systemic one, 6.4 km/s, utilizing system kinematic information used in the generation of the Keplerian mask and stacked to create a Keplerian deprojected spectrum for each transition.\textbf{ The radial range for generating the integrated spectrum is between 120 au and 550 au.} We have done a Gaussian fit to the observed spectrum using the \texttt{scipy.optimize.curve\_fit} to determine the peak intensity and the velocity at which the peak occurs. The width of the observed signal is full width at half maximum, given by 2\sqrt{2 \ln{2}}\sigma, where \sigma is the standard deviation calculated from our fit. We used \texttt{numpy.trapz()} function to calculate the disk-averaged integrated flux density of the line signal over a velocity range determined by visual inspection (the shaded region in Figure \ref{fig:int_spectra}). The Keplerian deprojected spectra are shown in the upper panel of Figure \ref{fig:int_spectra}. The stacked spectra for \nnhp and \dcop are peaking at the systemic velocity, v_{\text{LSRK}} of 6.4 km/s, confirming the detections. The disk-averaged integrated fluxes are reported in Table \ref{tab:obs_lines}. \textbf{The 1\sigma errors are calculated as \delta S \times \delta v \times \sqrt{N}, where \delta S is the noise associated with each channel, \delta v is the spectral resolution, and N is the number of channels over which integration is performed.}\par$
$\begin{deluxetable*}{lcccccccc}[!ht]$
$\tablecaption{Properties of the Lines Observed}$
$\label{tab:obs_lines}$
$\tablewidth{\textwidth}$
$\tablehead{$
$  \colhead{Species} & \colhead{Transition} & \colhead{Frequency} & \colhead{E_u} & \colhead{log_{10}(A_{ij})} & \colhead{g_u} & \colhead{Filter Response} & \colhead{Integrated Intensity} &$
$  \colhead{Line Width}\\[-1ex]$
$  \colhead  & \colhead  & \colhead{(GHz)} & \colhead{(K)} & \colhead{(s^{-1})} & \colhead  & \colhead{(\sigma)}$
$  & \colhead{(mJy km s^{-1})} & \colhead{(km s^{-1})}$
$}$
$\startdata$
$N_2H^+ & 4-3 & 372.6724808 & 44.71 & -2.50934 & 81 & 43 & 1570 \pm 40 & 0.55 \pm 0.01 \ DCO^+ & 5-4 & 360.1697783 & 51.86 & -2.42480 & 11 & 20 & 290 \pm 20 & 0.46 \pm 0.01 \ H_2D^+ & 1(1,0)-1(1,1) & 372.4213558 & 104.20 & -3.96567 & 9 & - & \textless 58 & - \ ^{13}CS & 8-7 & 369.9085505 & 79.89 & -2.97351 & 34 & - & \textless 70 & - \ SO_2 & 19(4,16)-19(3,17) & 359.7706846 & 214.26 & -3.41473 & 39 & - & \textless 35 & - \ \enddata$
$\tablecomments{$
$The error bars on the integrated intensity for the detected molecules are 1\sigma. We have calculated a 3\sigma upper limit on flux densities for the undetected molecules for an integration area between radius 150 au and 450 au and a line width of 0.55 km/s to get the integrated intensity values (Appendix \ref{sec:rotational_diagram}). The line width comes from the full-width half maximum of the Gaussian fitting in Figure \ref{fig:int_spectra} (Section \ref{subsec:obs_res}).}$
$\end{deluxetable*}$
$We further verified our detections with the matched filtering technique  (loomis2018detecting) . \textbf{In this method, we cross-correlate observed visibility with modelled visibility derived from the Keplerian masks used during cleaning.} We utilized a package called \texttt{VISIBLE}  (loomis2018visible)  for this task. For \nnhp, we found a filter response of 43 \sigma, and for \dcop, it is 20 \sigma at the systemic velocity of 6.4 km/s (Table \ref{tab:obs_lines}). The matched filter responses are shown in the Figure \ref{fig:matched_filter}.$
$\begin{figure*}[th!]$
$    \centering$
$    \includegraphics[width=\textwidth]{int_spetra.pdf}$
$    \caption{Keplerian deprojected and stacked spectra of the detected lines \nnhp and \dcop. The shaded region is the region over which we have done a Gaussian fitting and calculated integrated flux densities. The stacked spectra are peaking at the systemic velocity, v_{\text{LSRK}} of 6.4 km/s, confirming the detections. The location of the systemic velocity is shown by the vertical dashed lines. Integrated flux densities and matched filter responses are tabulated in Table \ref{tab:obs_lines}.}$
$    \label{fig:int_spectra}$
$\end{figure*}$
$Additionally, calibrated visibilities were exported to UVFITS format using the CASA task \texttt{exportuvfits} for subsequent analysis with the \textsc{Imager} program\footnote{\url{https://imager.oasu.u-bordeaux.fr}} and the \textsc{DiskFit} disk model fitting tool  ([Piétu, Dutrey and Guilloteau 2007]()) . Spectra and radial peak brightness profiles generated by \texttt{KEPLER} are presented in Figure \ref{fig:kepler-n2h} in Appendix \ref{app:imager}. The \texttt{KEPLER} command in \textsc{Imager} is similar to the \texttt{GoFish} package.$
$The reconstructed spectra and radial profiles slightly differ from those in Figure \ref{fig:int_spectra} because of the higher angular resolution (0.65" \times 0.53" at PA 80^\circ) used in \textsc{Imager}.$
$Both \nnhp and \dcop are detected with a very high signal-to-noise ratio, with \nnhp being the stronger detection among the two. \textbf{Radial profiles (Figures \ref{fig:radial_intensity}, \ref{fig:kepler-n2h}) indicate that the \nnhp emission exhibits a ring-like radial distribution, peaking near the outer edge of the ring. In contrast, the \dcop emission shows a double-ringed structure.} \textbf{The inner \dcop emission arises from the ring region, while an outer, brighter \dcop emission originates from$
$ the outer disk region.} This suggests radial variations in the deuteration process across the disk. Other targeted species, including H_2D^+, \ce{^{13}CS}, and SO_2, are not detected (Figure \ref{fig:matched_filter}).$
$\section{Derivation of Disk Physical Parameters}$
$\label{sec:constr_disk_phy_params}$
$\subsection{DiskFit modelling}$
$\label{subsec:diskfit}$
$\textbf{To derive disk properties, we perform a least-square fit to the observed visibilities to adjust an empirical parametric disk model to the data, using the \textsc{DiskFit} tool from [Piétu, Dutrey and Guilloteau (2007)]().} This approach allows an accurate derivation of the geometrical parameters. Furthermore, when several transitions of the same molecules are available, it allows a quantitative estimate of the molecular surface densities and excitation temperatures. It also allows for consistent estimates of the errors on the disk parameters (within the framework of the adopted disk model). This can be done either from the covariance matrix (when parameter coupling is limited) or through a more elaborate$
$Monte Carlo Markov Chain method.$
$The disk model is similar to that used in [Phuong, Dutrey and Diep (2020)](). It uses a flared disk geometry with a Gaussian vertical profile and a radial power-law for the scale height. Molecules are assumed to follow the same vertical distribution for simplicity. The model assumes level populations are governed by a Boltzmann law, with the temperature being a power law of radius. For a simple comparison with the \textbf{observed data}, we assume that the intrinsic line width remains constant with radius. \textbf{In fact, fitting line width as a power law instead suggests a small but non-significant decrease with radius that does not affect the other disk parameters.} The emerging line intensity is derived using simple ray tracing, considering the geometry and dynamics  (position, orientation, inclination, velocity, and Keplerian rotation pattern) and level populations computed at every sampled point along the line of sight. The hyperfine structure of the observed lines is also considered at this stage. This approach implicitly accounts for line optical depth.$
$Visibilities are then computed on the same (u,v) points as the data from the 3D data cube. A least square minimization is performed on the different visibilities using a modified Levenberg-Marquardt algorithm. \textbf{Guided by the radial distribution of velocity-integrated brightness (Figure \ref{fig:radial_intensity}) and apparent brightness temperature (Figure \ref{fig:kepler-n2h}), we represented the radial surface density profiles for each molecule by the sum of two Gaussians, truncated to inner and outer radii of 150 and 550 au, respectively. The rotation temperature is assumed to be a simple power law, newcommandined by its value at 250 au, T_{250} and exponent q.} The model thus has 15 possible parameters, 6 due to geometry and 9 to represent the molecular distribution and excitation conditions (see Tables \ref{tab:geom}-\ref{tab:diskfitabun} for their designation). Error bars were derived from the covariance matrix. The 6 geometric parameters, the line width \delta V, and temperature parameters exhibit very little coupling among them and with the other ones. However, the Gaussian distribution parameters are strongly coupled, particularly the pair (N_i, W_i) for distribution i, so error bars should be treated with caution.\par$
$The geometric parameters derived from N_2H^+ and DCO^+$
$are given in Table \ref{tab:geom}, using the distance$
$of 150 pc determined from Gaia. These transitions are relatively optically thin and, thus, more reliable tracers of the disk inclination than those previously used. Derived inclinations and orientation agree (but offer better precision) with previous determinations from CO isotopologues   ([Dutrey and Guilloteau 2014](), [Phuong, Chapillon and Majumdar 2018]()) .$
$\begin{deluxetable}{lrclrclc}$
$\tablecaption{Geometric and Dynamic parameters}$
$\tablewidth{\columnwidth}$
$\tablehead{$
$  \colhead{Quantity} & \colhead{N_{2}H^{+} (4-3)} &  \colhead{DCO^{+} (5-4)} &  Adopted$
$  }$
$\startdata$
$X_0 (") &   -0.007 \pm     0.011 &    0.019  \pm  0.016 &  0 \ Y_0 (") &   -0.001 \pm     0.007 &    0.004  \pm  0.011 & 0 \ PA (^\circ) &      8.4 \pm   0.5 &  8.1  \pm   0.5 & 8 \ i (^\circ) &     36.5 \pm  0.4 &  37.0  \pm  0.4 & 37 \ V_{sys} (km.s^{-1}) &  6.45 \pm  0.03 &     6.43 \pm  0.03 & 6.43 \ V_{100}  (km.s^{-1}) & 3.61 \pm  0.03 &     3.54 \pm  0.03 & 3.55$
$\enddata$
$\label{tab:geom}$
$\tablecomments{$
$\newline $
$Best fit geometric parameters from the observed visibilities. \textbf{Offsets (X_0,Y_0)} are from the ring center. V_{100} is the Keplerian velocity at 100 au. PA of 8^\circ in \textsc{DiskFit} is equivalent to 278^\circ in \texttt{GoFish}. In \textsc{DiskFit}, PA is measured with respect to the disk minor axis, while in \texttt{GoFish}, this angle is measured with respect to the disk major axis.}$
$\end{deluxetable}$
$To derive the surface densities and rotation temperatures, we used complementary data, \textbf{which is made of the interferometric visibilities obtained with NOEMA for}$
$N_2H^+ (1-0) and DCO^+ (1-0) from  [Phuong, Dutrey and Chapillon (2021)](), and the DCO^+ (3-2) from [Phuong, Chapillon and Majumdar (2018)](). Results are presented in Table \ref{tab:diskfitabun}. Figure \ref{fig:diskfit} shows the constrained radial distributions of surface densities. Agreement with the observations can be seen in Figure \ref{fig:kepler-n2h}, where the best fit radial profile {\textbf{(reconstructed by imaging the model visibilities and applying the Kepler deprojection)}is compared with the data. \par$
$While this process is appropriate for DCO^+,$
$the N_2H^+ distribution can almost equally well be represented by a single Gaussian or a truncated power law. This, however, does not affect the temperature derivation. The temperature that best represents N_2H^+ is almost constant, slightly increasing with radius and quite low, 12 K. On the contrary, DCO^+ is better represented by higher temperatures, decreasing with radius from \sim 18 K at 200 au to \sim 12 K at 500 au.\par$
$\textbf{As expected, the FWHM line width derived from the uv plane analysis with \textsc{DiskFit} (0.25  km/s using the 1/e values in Table \ref{tab:diskfitabun}) is smaller than those in the recentered spectra from \texttt{GoFish} (0.55 km/s, Fig.\ref{fig:int_spectra}) and \texttt{KEPLER} (0.37 km/s, Fig. \ref{fig:kepler-n2h}) because of the remaining Keplerian shear within the synthesized beams used in these image plane reconstructions.}$
$For H_2D^+, we simply assumed the same geometrical parameters, \textbf{a constant temperature of 15 K, and a uniform surface density between 150 and 450 au}. The best fit surface density of ortho-H_2D^+ from \textsc{DiskFit} is then (0.7 \pm 0.7) \times 10^{10} cm^{-2}. This gives the 3\sigma upper limit on o-\ce{H2D+} to be 2.1 \times 10^{10}cm^{-2}.$
$\begin{figure*}[!ht]$
$    \centering$
$    \includegraphics[width=\textwidth]{radial_colden_diskfit.pdf}$
$\caption{Radial distribution of surface density derived from the \textsc{DiskFit} modelling. The solid purple lines represent the total surface density while the dashed lines show the contribution from the individual Gaussians fitted in \textsc{DiskFit}. The grey horizontal lines indicate the derived disk-averaged values using the simple LTE approach.}$
$\label{fig:diskfit}$
$\end{figure*}$
$\begin{deluxetable}{lccl}[!hbt]$
$\tablecaption{surface density and Rotation temperatures}$
$\tablewidth{\columnwidth}$
$\tablehead{$
$  \colhead{Quantity} & \colhead{N_{2}H^+} & \colhead{DCO^{+}} &  Unit$
$  }$
$\startdata$
$\delta V &    0.149 \pm 0.005 &    0.150 \pm  0.005 & km s^{-1}\ T_{250} &     12.2 \pm  0.7  &     16.0  \pm  1.2   & K\ q       &    -0.32  \pm 0.19 &     0.39 \pm  0.15   & \ N_{1}   &   22 \pm 8    &    2.8 \pm 0.6  & 10^{11} cm^{-2} \ R_{1}   &    252 \pm   9 &    210 \pm  50  & au \ W_{1}   &     95 \pm 115 &     90 \pm 360  & au \ N_{2}   &   12 \pm   2   &   5.3  \pm 1.0  & 10^{11} cm^{-2}\ R_{2}   &    336 \pm 20 &    375 \pm 10  & au\ W_{2}   &    145 \pm 15 &     90 \pm 15  & au\ \enddata$
$\label{tab:diskfitabun}$
$\tablecomments{$
$\newline$
$Derived surface densities and temperatures in a double Gaussian radial distribution model.$
$N_{i} is the peak surface density of Gaussian distribution i, R_{i} its peak position, and W_{i} its width (FWHM). \delta V is the 1/e line width. T_{250} is the temperature at 250 au, q the exponent of the temperature power law \textbf{(a positive exponent indicates a quantity decreasing$
$with radius)}}.$
$\end{deluxetable}$
$\subsection{Results}$
$\label{sec:surfden_results}$
$\textbf{$
$Figure \ref{fig:diskfit} displays the radial distribution derived from the \textsc{DiskFit} modelling. The results from a classical LTE approach on disk averaged quantities (presented in  Appendix \ref{sec:rotational_diagram}) are also indicated. } The discrepancy between radial profile and disk averaged value for DCO^+ at large radii can be attributed to the assumption of constant T_{rot} = 16  K while the \textsc{DiskFit} model shows it drops to 12 K at 500 au, leading to a stronger Boltzmann distribution correction factor.$
$We did not detect the lines \hhdp, \CS, and \soo. We calculated upper limits on disk averaged surface densities for these molecules using Equation \eqref{eq:Nt_thin}, while assuming optically thin emission. The calculations considered 3\sigma values integrated over the line width of N_2H^+ (coming from Gaussian fit in Figure \ref{fig:int_spectra}). \textbf{Here, the noise \sigma = f \times \delta S \times \sqrt{\delta v \times \Delta v}, where f is the spectral correlation factor of 1.6, \delta S is the root mean squared noise in the flux, \delta v is the velocity resolution and \Delta v is the line width over which noise is calculated. The excitation temperature was fixed at 15 K for this purpose.$
$The integration area \Omega is considered for a radial region ranging from 150 au to 450 au. The constrained values are tabulated in Table \ref{tab:col_den}.}$
$\textbf{Our simple LTE approach reports an upper limit on total (summation of both ortho and para species) \ce{H2D+} surface density of 7.48 \times 10^{12} cm^{-2}, while our \ce{DiskFit} analysis quotes an upper limit on ortho-\ce{H2D+} surface density of 2.1 \times 10^{10} cm^{-2}. Converting this to a total H_2D^+ surface density requires an assumption on the ortho-to-para ratio. Under the hypothesis of thermalization, this ratio can be very small since the ortho-H_2D^+ ground state is 86 K above the para ground state, leading to a huge Boltzmann correction factor (from 70 to 300 for temperatures ranging from 20 down to 15 K respectively). At 15 K, the 3\sigma upper limit on the total \ce{H2D+} surface density, derived from the \ce{DiskFit} ortho-\ce{H2D+} value, is 6.3 \times 10^{12} cm^{-2}.}$
$\textbf{Note that the low-level detection of ^{13}CS (2-1) with NOEMA, a low energy transition reported by [Phuong, Dutrey and Chapillon (2021)](),  implies a surface density of 3 \times 10^{11} cm^{-2} for T_{rot} = 15 K, below our upper limit.}$
$\begin{deluxetable*}{lcccc}$
$\tablecaption{Disk averaged surface densities of the detected molecules and upper limits on the undetected molecules under LTE assumption}$
$\label{tab:col_den}$
$\tablewidth{\textwidth}$
$\tablehead{$
$  \colhead{Species} & \colhead{Transition} & \colhead{surface density, N_{T} (cm^{-2})} & \colhead{Excitation Temperature, T_{ex} (K)} & \colhead{Optical Depth, \tau}$
$}$
$\startdata$
$N_2H^+ & 4-3 & (1.19 \pm 0.04) \times 10^{12} & 12 (fixed) & 0.42 \ DCO^+ & 5-4 & (1.27 \pm 0.08) \times 10^{11} & 16 (fixed) & 0.04 \ H_2D^+ & 1(1,0)-1(1,1) & <7.48 \times 10^{12} & 15 (fixed) & -  \ ^{13}CS & 8-7 & <1.12 \times 10^{12} & 15 (fixed) & -  \ SO_2& 19(4,16)-19(3,17) & <2.46 \times 10^{16} & 15 (fixed) & - \ \enddata$
$\tablecomments{$
$\newline$
$a) \textbf{Surface density error are coming from 16th and 84th percentile of the posterior distribution.}\ b) The upper limits on the disk averaged surface densities of the undetected molecules (H_2D^+, ^{13}CS and SO_2) are constrained using the  3\sigma upper limit on the line flux as explained in Section \ref{sec:rotational_diagram}.\ c) \textbf{In this table, we have reported the upper limit on the total (sum of ortho and para spin-states) \ce{H2D+} surface density. The 3\sigma upper limit for o-\ce{H2D+} from \textsc{DiskFit} analysis is 2.1 \times 10^{10} cm^{-2}.}$
$}$
$\end{deluxetable*}$
$\section{Comparison of Observations with Models} \label{sec:models}$
$\subsection{Disk Physical Model} \label{subsec:phy_model}$
$Our physical model is constructed for a radial range of 190 au to 290 au, which primarily includes the ring region around the central stellar system GG Tau A. Only 10-20\% of the total disk mass is contained in the outer disk region, which, therefore, exhibits a lower density than in the ring. Moreover, the outer disk, under the shadow of the dense ring, is very cold  ([Tang, Dutrey and Guilloteau 2016](), [Dutrey and Guilloteau 2014](), [Brauer and Pantin 2019]())  with poor constraints on the physical properties. We chose not to focus our present modelling effort on this outer region, which has a more complex structure  ([Tang, Dutrey and Guilloteau 2016](), [Phuong, Dutrey and Diep 2020]()) .\par$
$In the ring, the radial distribution of the midplane (T_{mid}) and the atmospheric temperature (T_{atm}) are taken as power laws, constrained in previous studies by [Dutrey and Guilloteau (2014)]() and [Guilloteau, Dutrey and Simon (1999)]() respectively. They are of the form$
$\begin{align}$
$	T_{mid} (r) &= T_{mid, R_{\text{ref}}} \left( \frac{r}{R_{\text{ref}}} \right)^{-q} \label{eq:T_mid}\ 	T_{atm} (r) &= T_{atm, R_{\text{ref}}} \left( \frac{r}{R_{\text{ref}}} \right)^{-q}\label{eq:T_atm}$
$\end{align}$
$The values of the parameters can be found in Table \ref{tab:physical_model}. The radial dependency of  scale height, H_r \textbf{at the disk midplane} calculated assuming hydrostatic equilibrium using the midplane temperature (Equation \ref{eq:T_mid}), the mass of the stellar system, M_* = 1.37 M_{\odot} (rescaled from [Guilloteau, Dutrey and Simon (1999)]() considering a distance of 150 pc), the mean molecular weight of the gas per H nuclei \mu = 2.37 and the atomic mass unit, m_H, as follows$
$\begin{align}$
$	H_r= \sqrt{\frac{k_B T_{mid} r^3}{\mu   m_H   G M_*}} \label{eq:H}$
$\end{align}$
$Here k_B and G are Boltzmann and Gravitational constants, respectively. We constructed the 1D vertical structure at each radial point until 4H_r height. The vertical temperature structure is adopted from [Dartois, Dutrey and Guilloteau (2003)]() and Williams2014 given by,$
$\begin{align}$
$    T(z) = T_{mid} + (T_{atm} - T_{mid})\left[ sin \left(\frac{\pi z}{2 z_q}\right)\right]^{2\sigma}$
$\end{align}$
$T_{mid} and T_{atm} are newcommandined at the midplane (z = 0) and the upper end (z = z_q) of the atmosphere assuming Equations \ref{eq:T_mid} and \ref{eq:T_atm}, respectively. z_q is set at 4H_{r} in our case. The parameter \sigma denotes the stiffness of the vertical temperature distribution. As pointed out by [Tang, Dutrey and Guilloteau (2016)](), the inner edge of the GG Tau ring obstructs a substantial amount of starlight, minimizing the vertical temperature variation within. Therefore, we have chosen a value of 0.5 for \sigma. We have kept the dust temperature equal to the gas temperature, as dust and gas are relatively well coupled for such densities \textbf{(>10^8 cm^{-3})}. \par$
$The gas surface density, \Sigma_g(r) [g cm^{-2}] is parameterized as a power law profile suggested by [Lynden-Bell and Pringle (1974)](), given by:$
$\begin{align}$
$	\Sigma_g(r) = \Sigma_{g, R_{\text{ref}}} \left( \frac{r}{R_{\text{ref}}} \right)^{-\gamma} \label{eq:sigmag_r}$
$\end{align}$
$The power law index \gamma is taken to be 1.4 following [Dutrey and Guilloteau (2014)](). The gas surface density at the reference radius R_{\text{ref}}, \Sigma_{g, R_{\text{ref}}} is calculated considering the mass contained in the circumtertiary ring, M_{g} = 0.13 M_{\odot}  ([Guilloteau, Dutrey and Simon 1999](), [Andrews, Chandler and Isella 2014]()) . The calculations are as follows:$
$\begin{align}$
$	M_g = 2\pi \int_{R_{in}}^{R_{out}} \Sigma_g(r)rdr\label{eq:Mg}$
$\end{align}$
$Substituting Equation \ref{eq:sigmag_r} in \ref{eq:Mg}, for R_{\text{ref}}=214 au, we get:$
$\begin{align}$
$	\Sigma_{g, R_{\text{ref}}} = \frac{(2-\gamma) M_g}{2 \pi R_{\text{ref}}^\gamma} \left[ R_{\text{out}}^{2-\gamma} - R_{\text{in}}^{2-\gamma}\right]^{-1} = 9.7 \text{ g cm^{-2}}$
$\end{align}$
$We then calculate the \ce{H2} number density [cm^{-3}] at the midplane with,$
$\begin{align}$
$    n_{H_2,midplane} = \frac{\Sigma_g(r)}{\mu  m_H H_r\sqrt{2\pi}}$
$\end{align}$
$Using the midplane density as the lower limit, we integrate the equation of hydrostatic equilibrium to obtain the vertical density structure  ([Reboussin, et. al 2015]()) :$
$\begin{align}$
$    \frac{\partial \ln{n_{H_2}(z)} }{\partial z} = - \left[ \left( \frac{G M_* z}{r^3} \right) \left( \frac{\mu m_H}{k_B T} \right) + \frac{\partial \ln{T}}{\partial z}\right]$
$\end{align}$
$The local visual extinction is determined from hydrostatic density structure, assuming a conversion factor of (A_v/N_H)_0 = 6.25 \times 10^{-22}  (Wagenblast1989)  where N_H = 2N_{H_2} is the vertical hydrogen surface density. \textbf{We assume N_H, and consequently A_v are zero above 4H_r.} To account for the impact of grain size (r_d) and dust-to-gas mass ratio (\varepsilon), we scale the conversion factor as follows  ([Wakelam, Chapillon and Dutrey 2019]()) :$
$\begin{align}$
$    \frac{A_v}{N_H} = \left( \frac{A_v}{N_H} \right)_0 \frac{\varepsilon}{10^{-2}} \frac{10^{-5}}{r_d(cm)}$
$\end{align}$
$\textbf{The UV flux factor f_{\text{UV}} follows the inverse squared law of distance and bombards at the top of the disk atmosphere, only half of which is scattered towards the midplane, the rest being absorbed or scattered upwards  ([Wakelam, Ruaud and Hersant 2016]()) }:$
$\begin{align}$
$    f_{\text{UV}} (r) = \frac{1}{2}\frac{f_{\text{UV}, R_{\text{ref}}}}{\left( \frac{r}{R_{\text{ref}}}\right)^2 + \left( \frac{4H_r}{R_{\text{ref}}} \right)^2}$
$\end{align}$
$f_{\text{UV}, R_{\text{ref}}} is newcommandined at the surface of the disk in the units of interstellar DRAINE field spectrum and reported in Table \ref{tab:physical_model}. H_r is calculated using Equation \ref{eq:H}. \textbf{The vertical distribution of UV flux is obtained by multiplying f_{UV} with e^{-A_v/1.086}, where A_v is the visual extinction distribution  ([Du and Bergin 2014]()) .}\par$
$\textbf{In our models, we have not implemented grain growth; instead considered a uniform grain size of 0.1\mu m  ([Phuong, Chapillon and Majumdar 2018]())  settling below 1H_r.} A single-grain size is justified under the assumption that small grains dominate the surface area throughout the disk and are the most important for chemistry.$
$To incorporate dust settling, we adjust the \varepsilon to 10^{-3} above 1H_r, while maintaining the standard \varepsilon of 10^{-2} below 1H_r. Figure \ref{fig:best_fit_structure} illustrates the corresponding structure of the best physical model.$
$\begin{deluxetable}{lcc}[!t]$
$\tablecaption{Parameter prescription of the best-fit model considered\label{tab:physical_model}}$
$\tablehead{$
$\colhead{Parameter Description} & \colhead{Values} & \colhead{Units}$
$}$
$\startdata$
$\cutinhead{Fixed Parameters}$
$Stellar mass: M_* & 1.37 & M_\odot\ Ring mass: M_g & 0.13^a & M_\odot\ Ring inner radius: R_{\text{in}} & 193 & au\ Ring outer radius: R_{\text{out}} & 285 & au\ Reference radius: R_{\text{ref}} & 214 & au \ Midplane temperature at R_{\text{ref}}: T_{\text{mid, R_{\text{ref}}}} & 14 & K\ Atmospheric temperature at R_{\text{ref}}: T_{\text{atm, R_{\text{ref}}}} & 30 & K\ Surface density at R_{\text{ref}}: \Sigma_{g,\text{ref}} & 9.7 & g cm^{-2}\ Temperature power-law index: q & 1.1 & \ Temperature vertical gradient index: \sigma & 0.5 & \ Surface density power law index: \gamma & 1.4 & \ \cutinhead{Adjusted Parameters}$
$UV Flux: f_{\text{UV}, R_{\text{ref}}}  & 375 & Draine's unit\ Cosmic ray ionization, \zeta_{CR} & 10^{-18} & s^{-1}\ Grain growth & No & \ Dust settling & Yes & \ C/O ratio & 1.0 &\ Age of the parent molecular cloud  & 10^6 & yr\ Age of the disk & 10^6 & yr$
$\enddata$
$\tablecomments{$
$All the fixed values are taken from [Guilloteau, Dutrey and Simon (1999)](), [Dutrey and Guilloteau (2014)]() and ^a[Andrews, Chandler and Isella (2014)]() and rescaled considering a distance of 150 pc, except \sigma and \Sigma_{\text{ref}}. \sigma is expected to be small as the inner edge of the ring casts a shadow on the rest of the structure  ([Tang, Dutrey and Guilloteau 2016]()) . \Sigma_{\text{ref}} is calculated using Equation \ref{eq:sigmag_r} assuming 0.13M_{\odot} of total disk mass is contained in the ring region.$
$}$
$\end{deluxetable}$
$\begin{figure*}$
$    \centering$
$    \includegraphics[width=\textwidth]{StructureD2_1e18_co1.0.pdf}$
$    \caption{Physical structure of the ring region in our model derived using the parameters tabulated in Table \ref{tab:physical_model}. The left one indicates the temperature distribution, the middle one is the gas density distribution in the log scale, and the one on the right is visual extinction in the log scale. The description of the model is explained in Section \ref{sec:models}.}$
$    \label{fig:best_fit_structure}.$
$\end{figure*}$
$\subsection{Astrochemical Model}\label{subsec:astrochem_model}$
$The gas-grain astrochemical model \textsc{Dnautilus}, introduced in [Majumdar, Gratier and Ruaud (2017)](), is used to compute 1+1D-chemical composition in the ring of GG Tau A. \textsc{Dnautilus} has the capability to investigate deuterium fractionation in both two- (gas and grain surface) and three-phase (gas, grain surface, and grain bulk) modes.$
$In this study, we employed the three-phase edition of \textsc{Dnautilus} to solve time-dependent chemical abundances utilizing the \texttt{kida.uva.2014} chemical network with updated deuterium chemistry  ([Taniguchi, Rayalacheruvu and Yonetsu 2024]()) . We have considered 37,336 gas-phase reactions, 7,675 grain-surface reactions, and 3,110 grain-mantle reactions among 896 gas species, 453 grain-surface species, and 453 grain-mantle species in our chemical network.\par$
$\subsection{Grid of Models}$
$\textbf{To find the model that best represents our observationally constrained radial surface density distribution of \ce{N2H+}, \ce{DCO+} and upper limits on \ce{H2D+}, \ce{SO2} and ^{13}CS \citep[including the constraint from the transition J=2-1 from][]{Phuong2021}, we explored a large grid of models. These models span a range of initial carbon-to-oxygen (C/O) ratios, cosmic ray ionization (CRI) rates and UV flux values (see Table \ref{tab:dnautilus_parameters} in Appendix \ref{app:chemical_model}).}$
$\textbf{The model with the least disagreement with observations is an inheritance disk model with initial abundances characterized from a starless dense molecular cloud, illuminated by stellar UV radiation of 375\chi_0 at a reference radius of 214 au.} First, we allowed the molecular cloud to evolve chemically for 10^6 years from initial atomic abundances (Table \ref{tab:initial_abun}) with typical physical conditions such as gas and dust temperature of 10 K, a total gas density of 2\times10^4 cm^{-3}, a visual extinction of 15 mag and cosmic ray ionization \textbf{rate} of 1.3 \times 10^{-17} s^{-1}. We then computed the temporal evolution of chemistry for the inherited material from the cloud in the ring region around GG Tau A for 10^6 years. \textbf{The assumption that protoplanetary disks form from a cold, starless dense molecular cloud with the mentioned physical conditions is based on detailed discussions from [Wakelam, Chapillon and Dutrey (2019)](), without taking into account the chemical evolution between the cold core phase and the protoplanetary disk itself  ([Drozdovskaya and Walsh 2016]()) . It is crucial to highlight that in order to reproduce the observed surface densities in our disk simulation, a cosmic ray ionization rate (\zeta_{\text{CR}}) of about 10^{-18} s^{-1} and an initial C/O ratio of 1.0 are necessary.} Table \ref{tab:physical_model} presents the parameters describing the best-fit model. Figure \ref{fig:modelled_abundances} showcases the modelled number density distributions within the ring region.$
$\begin{deluxetable}{lcc}$
$\tablecaption{Initial abundances used in our cloud model\label{tab:initial_abun}}$
$\tablewidth{\columnwidth}$
$\tablehead{$
$\colhead{Element} & \colhead{Abundance relative to H} & \colhead{References}$
$}$
$\startdata$
$\ce{H2}  & 0.5 &  \ He  & 9.00 \times 10^{-2} & 1 \ N   & 6.20 \times 10^{-5} & 2 \ O   & (1.40 - 3.3) \times 10^{-4} ^{(a)} & 3 \ \ce{C+}  & 1.70 \times 10^{-4} & 2 \ \ce{S+}  & 8.00 \times 10^{-8} & 4 \ \ce{Si+} & 8.00 \times 10^{-9} & 4 \ \ce{Fe+} & 3.00 \times 10^{-9} & 4 \ \ce{Na+} & 2.00 \times 10^{-9} & 4 \ \ce{Mg+} & 7.00 \times 10^{-9} & 4 \ \ce{P+}  & 2.00 \times 10^{-10} & 4 \ \ce{Cl+} & 1.00 \times 10^{-9} & 4 \ F   & 6.68 \times 10^{-9} & 5 \ HD  & 1.60 \times 10^{-5} & 6 \ \enddata$
$\tablerefs{ (1) [Wakelam and Herbst (2008)](); (2) [ and Jenkins (2009)](); (3) [Reboussin, et. al (2015)]() (4) Low metal abundances from [Graedel, Langer and Frerking (1982)](); (5) Depleted value from [Neufeld, Wolfire and Schilke (2005)](); (6) [Majumdar, Gratier and Ruaud (2017)](). (a) O initial abundance is adjusted for C/O ratio ranging from 0.5 to 1.2}$
$\end{deluxetable}$
$\begin{figure*}$
$    \centering$
$    \includegraphics[width=0.9\textwidth]{molecular_ndensD2_1e18_co1.0.pdf}$
$    \caption{Number density distributions (log-scale) from our best-fit model for \ce{CO}, N_2H^+, H_2D^+ and DCO^+ from left to right and top to bottom. We have only modelled the ring region around GG Tau A, spanning a radial distance of 193 au to 285 au from the central stellar system. The model disk atmosphere extends up to 4 scale heights.}$
$    \label{fig:modelled_abundances}$
$\end{figure*}$
$\begin{figure*}$
$    \centering$
$    \includegraphics[width=\linewidth]{colden_cri_1.pdf}$
$    \caption{Comparison between models with different CR rates. Best model: CRI= 10^{-18} s^{-1}. Here, C/O = 0.7. ^{13}CS surface density is calculated from CS surface density by assuming N_{\text{CS}}/N_{^{13}\text{CS}} = 100  ([Phuong, Dutrey and Chapillon 2021]()) }$
$    \label{fig:comp_cr}$
$\end{figure*}$
$\begin{figure*}$
$    \centering$
$    \includegraphics[width=\linewidth]{colden_co_1.pdf}$
$    \caption{Comparison between models with different initial C/O ratios. Best model: C/O= 1.0. Here, CRI = 10^{-18} s^{-1}. ^{13}CS surface density is calculated from CS surface density by assuming N_{\text{CS}}/N_{^{13}\text{CS}} = 100  ([Phuong, Dutrey and Chapillon 2021]()) }$
$    \label{fig:comp_co}$
$\end{figure*}$
$\section{Discussion} \label{sec:discussions}$
$\subsection{Comparison with Previous Observations}$
$\textbf{\hhdp has been observed in multiple dense starless cores and prestellar cores \citep[see for a review]{Ceccarelli2014}.$
$[Caselli, Vastel and Ceccarelli (2008)]() reports o-\ce{H2D+} surface density \sim 10^{12} - 10^{13} in regions characterized by gas density \sim 10^{5} - 10^{6} cm^{-3} and kinetic temperatures around 10 K, with most prominent detections in the densest and centrally concentrated cores. However, no clear detection has been obtained in protoplanetary disks. [Chapillon, et. al (2011)]() derived 3 \sigma upper limits on o-\ce{H2D+} surface density for the T Tauri disks DM Tau and TW Hya. Depending on the physical model, DM Tau's upper limit lies between 4.5 \times 10^{11} and 1.9 \times 10^{12} cm^{-2} and for TW Hya, it is between 9.0 \times 10^{11} and 1.4 \times 10^{12} cm^{-2}. Our upper limit for o-\ce{H2D+} in GG Tau is 2.1 \times 10^{10} cm^{-2}. Note that [Chapillon, et. al (2011)]()'s upper limits are from single-dish observations (1\sigma noise level 0.33 Jy km s^{-1}). We have achieved a much higher sensitivity (1\sigma noise level 0.019 Jy km s^{-1}), making our observation the best available constraint on the upper limit of o-\ce{H2D+} surface density in any disk to date.}$
$\textbf{The \ce{N2H+} (3-2) transition has been observed in all five disks (three around T Tauri stars: IM Lup, GM Aur, and AS 209, and two around Herbig Ae stars: HD 163296 and MWC 480) as part of the Molecules with ALMA at Planet-forming Scales (MAPS) program  ([Cataldi, Yamato and Aikawa 2021]()) . Despite the diverse continuum structures of the disks with multiple rings and gaps, \ce{N2H+} emission showcases a ring morphology (Figure 4 in [Cataldi, Yamato and Aikawa (2021)]()). Our \ce{N2H+} emission is also characterized by a single-ring structure (Figure \ref{fig:radial_intensity}). [Qi, Öberg and Wilner (2013)]() has previously reported \ce{N2H+} detection towards TW Hya, and the single-ring emission structure is also evident there. The \ce{N2H+} ring morphology across different disks can be attributed to radial and vertical temperature variations  ([Qi, Öberg and Wilner 2013]()) . In the MAPS survey by [Cataldi, Yamato and Aikawa (2021)](), two different \ce{N2H+} surface density values are derived by fixing the excitation temperature, T_{\text{ex}}, to a) the observationally constrained CO freeze-out temperature \sim 20 K and b) the midplane temperatures, T_{\text{mid}}, of the respective disks. For the T Tauri disks (IM Lup, GM Aur, and AS 209), the \ce{N2H+} surface density peaks around 10^{13} cm^{-2}, while for the Herbig Ae disks (HD 163296 and MWC 480), it reaches up to 10^{12} cm^{-2}. The surface density in TW Hya reported by [Qi, Öberg and Wilner (2013)]() ranges from 4 \times 10^{12} to 2 \times 10^{15} cm^{-2} for different assumed physical structures.[Cataldi, Yamato and Aikawa (2021)]() indicates that temperatures as low as 12 K in the \ce{N2H+} emitting region can also explain the observed flux of the 3-2 transition in the protoplanetary disks (e.g., GM Aur, HD 163296, MWC 480) under the LTE assumption. Previously, in [Phuong, Dutrey and Chapillon (2021)]()'s study, \ce{N2H+} in GG Tau A was assumed to be arising from the same layer as CO, following a radial temperature variation, T(r) = 20 (r/250 \text{ au})^{-1} K. Our direct measure of T(r) = 12 K for the \ce{N2H+} emitting region (Table \ref{tab:diskfitabun}), based on multi-line excitation analysis, suggests that we are tracing a region much closer to the midplane than initially assumed.}$
$\textbf{A ring-like radial distribution for \ce{DCO+} has been commonly observed in multiple disks, including the TW Hya, DM Tau, AS 209, V4046 Sgr, MWC 480 and HD 163296  ([Qi, et. al 2008](), [Mathews, Klaassen and Juhász 2013](), [Teague, Semenov and Guilloteau 2015](), [Huang, Öberg and Qi 2017]())  and is facilitated by reaction between \ce{H2D+} and \ce{CO} (e.g. [ and Willacy (2007)]()). Contrary to the common scenario, we observe partially overlapping double-ring radial distribution of \ce{DCO+} emission in GG Tau A (refer to Figure \ref{fig:radial_intensity}), which can likely be attributed to different chemical effects in the ring and the outer disk regions owing to the difference in densities between the two regions. This is discussed in detail in Sections \ref{subsec:chem_ring} and \ref{subsec:chem_outerdisk}. Notably, [Öberg, Furuya and Loomis (2015)]() observes double \ce{DCO+} rings in the disk IM Lup. They found that \ce{DCO+} abundance first decreases radially due to CO depletion, then another ring forms due to non-thermal desorption of CO in low surface density outer regions as a result of higher UV penetration. [Phuong, Chapillon and Majumdar (2018)](), [Phuong, Dutrey and Chapillon (2021)]() have previously assumed that \ce{DCO+} emission originates from the same region as \ce{N2H+} and \ce{CO} characterized by radial temperature, T(r) = 20 (r/ 250 \text{ au} )^{-1} K in GG Tau A. In this work, we found that \ce{DCO+} emission region is best represented by T(r) = 16 (r/250 \text{ au})^{-0.39} K (Table \ref{tab:diskfitabun}). Hence, we expect that \ce{DCO+} emission region lies vertically between \ce{CO} emitting region and \ce{N2H+} emitting region and closer to the midplane than previously thought.}$
$\subsection{Chemical Trends in the Ring Region} \label{subsec:chem_ring}$
$\textbf{We present a sub-set of chemical models for a selected set of input parameters in Figures \ref{fig:comp_cr} and \ref{fig:comp_co} from the full model grids presented in Table \ref{tab:dnautilus_parameters}. Only the CRI rates and the C/O ratio were found to have any significant impact on the modelled surface densities of the targeted molecules. Other parameters cannot be constrained by the current data.}$
$\paragraph{\textbf{Cosmic Ray Ionization}}$
$\textbf{Figure \ref{fig:comp_cr} shows the variation of modelled abundances with CRI rates while keeping the rest of the parameters fixed to standard values. \ce{N2H+}, \ce{DCO+}, and \ce{H2D+} abundances are quite sensitive to CRI rates, while UV flux values have no impact whatsoever. This is expected because these molecules probe a high-density, low-temperature, CO-depleted region near the mid-plane where UV penetration is minimal. Hence, cosmic rays are the primary ionization source for this cold molecular chemistry. However, the CRI rate has barely any impact on S-bearing species. It is worth noting that the standard interstellar CRI rate of 10^{-17} s^{-1} predicts the \ce{H2D+} surface density above the upper limit and a sub-interstellar CRI rate of 10^{-18} s^{-1} explains the observationally constrained surface densities the best. This low CRI rate can be explained by a rise in the number of collisions experienced by CR particles due to the greater effective surface density as they gyrate towards the disk midplane  ([Padovani, Hennebelle and Galli 2013]()) . The attenuation is especially prominent in our case, as our chemistry focuses on high-density near midplane regions. [Cleeves, et. al (2015)]() have previously inferred a very low CRI  (10^{-19} s^{-1}) in the protoplanetary disk TW Hya, which they suggest is due to an exclusion by a young stellar wind. Our current astrochemical model only includes a constant CR ionization rate across the disk. The inclusion of short-lived radionuclides  ([Cleeves, Adams and Bergin 2013]())  and vertical distribution of CRI rates  ([Padovani, et. al 2018]())  may new a better picture.}$
$\paragraph{\textbf{C/O ratio}}$
$\textbf{We have also modelled abundances of our targeted molecules for different elemental C/O ratios from 0.5 to 1.2 by changing initial O abundances, as shown in Figure \ref{fig:comp_co}. Changing the C/O ratios from 0.5 to 1.2 has decreased the \ce{N2H+} surface density by a factor of approximately 2.5, while the \ce{DCO+} surface density increases by at most a factor of 1-2, and the \ce{H2D+} surface density remains unaffected. Overall, the C/O ratio has an insignificant impact on these three molecules. However, the chemistry of \ce{^{13}CS} and \ce{SO2}, for which we obtained the upper limit from our observations, is quite sensitive to changes in the C/O ratio. The general trend is that O-poor chemistry favours the formation of the C-S bond, and for \ce{SO2}, it is the other way around$
$\citep[see discussions by][for further details]{Semenov+2018,LeGal2021}.$
$Our observations are best represented by a CRI rate of 10^{-18} s^{-1} and an elemental C/O ratio of 1.0. At the commencement of the evolution of a molecular cloud, a significant amount of important elements, including carbon (C) and oxygen (O), exist in a refractory state, which is expected to shape the chemical budget of the protoplanetary disk formed from that cloud. Based on [Boogert, Gerakines and Whittet (2015)]()'s observation on ice abundances, [Öberg and Bergin (2021)]() reports that about 40\% O chemical budget was unaccounted for at the onset of star and planet formation, leading to a poor constraint on initial C and O gas phase abundances. Our elemental C/O ratio of 1.0 suggests an oxygen-depleted environment in the dense ring.$
$Similar inferences of high C/O ratios (0.9-2.0) were also found in more typical protoplanetary disks  \citep[e.g.][]{Ruaud2022, LeGal2021}, which may have consequences on the formation of planets in the inner regions.}$
$\paragraph{\textbf{Spatial Distributions}}$
$\textbf{Figure \ref{fig:modelled_abundances} represents the molecular number density distribution for our best model. We see a \ce{DCO+} distribution in those regions where CO is available in the gas phase in small amounts. Observationally, \ce{DCO+} follows a ring-like emission distribution in the ring region (refer to Figures \ref{fig:radial_intensity} and \ref{fig:kepler-n2h}). In our modelled abundances, \ce{DCO+} is primarily peaking up in temperature regions representative of the warm molecular layer (2.5- 3.5H_r), somewhat showcasing the ring-like distribution. One of the dominant ions in the warm molecular layer is \ce{HCO+}. It essentially forms through the protonation of \ce{CO} and, in turn, produces DCO^+ through isotope exchange. The major reaction for DCO^+ production contributing to our observation above 2.5H_r is \ce{HCO+ + D -> DCO+ + H}. We also see some \ce{DCO+} arising below the warm molecular layer (\sim 0.5 - 2.5 H_r) produced through a different channel, \ce{CO + N2D+ -> N2 + DCO+}. Our \textsc{DiskFit} analysis locates the peak of the \ce{DCO+} ring at radial distance 210 \pm 50 au (R_1 in Table \ref{tab:diskfitabun}), and this can be attributed to the fact that both the \ce{DCO+} formation channels are contributing to \ce{DCO+} abundances in this radial range. Near the disk midplane, \ce{DCO+} starts depleting because its parent molecule, \ce{CO}, efficiently freezes onto the grain surfaces owing to high density and low temperature.}$
$\textbf{\ce{N2H+} modelled distribution lies just below the \ce{DCO+} layer, with its number density peaking up in the regions where \ce{DCO+} starts depleting. Our observationally constrained representative temperatures of 16 K and 12 K for \ce{DCO+} and \ce{N2H+} emitting regions, respectively (Table \ref{tab:diskfitabun}), suggest the same. N_2H^+ is abundant in the midplane at the inner edge of the ring as well as in the warm molecular layer (up to 3.0H_r) at the outer edge of the ring. All throughout the ring region, the major formation pathway of N_2H^+ is the protonation of \ce{N2}, i.e., \ce{N2 + H3+ -> H2 + N2H+}. \ce{H3+} is the most abundant ion in the near-midplane region. So, the chemistry is mainly controlled by gas phase abundances of \ce{CO}, \ce{N2} and \ce{HD}. \ce{CO} is the most efficient in capturing \ce{H3+}, followed by \ce{N2} and \ce{HD}. As long as \ce{CO} remains in the gas phase, it destroys \ce{N2H+} to form \ce{HCO+}. When CO starts depleting, \ce{N2H+} start increasing, as \ce{N2} is still in the gas phase owing to its lower binding energy with respect to CO, forming \ce{N2H+} and there is less destruction of \ce{N2H+} because of the reduced CO presence.}$
$\textbf{Our astrochemical model suggests that \ce{H2D+} is abundant in the midplane region, situated just below the \ce{N2H+} layer. The major formation pathway predicted by our model is \ce{H3+ + HD -> H2D+ + H2}. As \ce{N2} begins to freeze onto grain surfaces, more \ce{H3+} become available for \ce{HD} to capture, and we see a rise in \ce{H2D+} abundances and a fall in \ce{N2H+} abundances.}$
$We did not detect \ce{H2D+}. In our effort to replicate the observed surface density for the detected molecules, we realized that a lower cosmic ray ionization rate may new the best explanation for our observations. \ce{H2D+} in the cold midplane region is primarily formed by protonation of \ce{HD}. Therefore, \ce{H2D+} abundances are regulated by the availability of \ce{H3+}, which in turn is dependent on the rate of ionization of \ce{H2}. A lower CRI rate results in reduced \ce{H2+} production (\ce{H2 + CR -> H2+ + e-}), leading to a lower abundance of \ce{H3+} (\ce{H2 + H2+ -> H3+ + H }). Hence, a lower ionization rate at the midplane, owing to the very high density in the ring region, may explain the non-detection of \hhdp. Indeed, our model using a CR rate (\zeta_{CR}) of 10^{-18} s^{-1} predicts a \ce{H2D+} surface density of the order of \sim 10^{12} cm^{-2} close to the current upper limit reported in Table \ref{tab:col_den}. This limit suggests we are close to detection, and increasing the observation time should assure our chance of detection for \ce{H2D+}. However, the upper limit on total \ce{H2D+} is based on the assumption of thermalized ortho-to-para ratio (LTE), with a high correction Boltzmann factor (see Section \ref{sec:surfden_results}).$
$While thermalization is facilitated by the very high mid-plane density, spin-dependent reaction rates may lead to a different prediction \citep[e.g.][]{Chapillon2011}.$
$\subsection{Possible Chemistry in the Outer Disk}\label{subsec:chem_outerdisk}$
$\nnhp and \dcop emission are also spatially extended beyond the ring, reaching up to a radial distance of approximately 550 au (Figures \ref{fig:mo_maps}, \ref{fig:radial_intensity}). The velocity-integrated radial profiles of \nnhp and \dcop (Figure \ref{fig:radial_intensity}) indicate that the former is decreasing as we go radially outward in the outer disk, while the latter shows another peak at around \sim 375 au.\par$
$We have not modelled the outer disk due to its complex structure  ([Tang, Dutrey and Guilloteau 2016](), [Phuong, Dutrey and Diep 2020]()) . However, based on our current understanding, the outer disk is characterized by lower density and colder temperatures as compared to the ring (refer to Section \ref{subsec:phy_model}). Lower densities outside the dense ring allow higher CR penetration, leading to non-thermal desorption of CO and \ce{N2}. Consequently, an increased presence of CO and \ce{N2} in the gas phase is anticipated in the outer disk, suggesting that the primary pathway for DCO^+ production would be through \ce{H2D+ + CO}  ([Phuong, Chapillon and Majumdar 2018]()) . On the other hand, \ce{N2H+} formed through protonation of \ce{N2} will be readily destroyed in the presence of \ce{CO}, which will cause a rapid decrease in the \ce{N2H+} abundances as seen in Figure \ref{fig:radial_intensity}.$
$\subsection{Temperature Trends in the Ring Region}$
$The \textsc{DiskFit} analysis of our observations of \nnhp and \dcop along with NOEMA observations N_2H^+ (1-0) and DCO^+ (1-0, 3-2) by [Phuong, Dutrey and Chapillon (2021)](), [Phuong, Chapillon and Majumdar (2018)]() indicate that N_2H^+  and DCO^+ are best represented by average temperatures of 12 K and 16 K (Table \ref{tab:diskfitabun}) respectively. These rotation temperatures are consistent with N_2H^+ being located closer to the cold disk mid-plane than DCO^+ and a fortiori, CO and its isotopologues previously studied by [Phuong, Dutrey and Diep (2020)]().\par$
$\textbf{However, in our chemical model, while the distribution of \ce{N2H+} and \ce{DCO+} extends to regions with temperatures of 12 and 16 K, their density-weighted average temperatures are 21 K and 24 K, respectively. The discrepancy with observed temperatures can be attributed to our limited understanding of the physical structure of the ring region and the simplifications applied in the model to derive the physical structure}. DCO^+ abundances are sensitive to grain size, and the dust-to-gas mass ratio significantly influences gas-phase molecular abundances. [Gavino, Dutrey and Wakelam (2021)](), [Gavino, Kobus and Dutrey (2023)]() recently investigated the impact of the dependence of the grain size on their temperature. They found that large grains exhibit lower temperatures than smaller ones, \textbf{with an effect on chemistry that cannot be properly represented by a single grain size. This can influence the structure of the CO distribution (with more CO near the mid-plane) and, consequently, the gas-phase abundances of DCO^+ and N_2H^+ in the densest, coldest regions.}$
$\section{Conclusions} \label{sec:conclusion}$
$In this paper, we have presented high-sensitivity ALMA band 7 observations of the circumstellar disk around the triple star system GG Tau A \textbf{that highlights CO-regulated cold molecular chemistry in the shielded high-density regions near the disk midplane.} Our key findings and conclusions are summarized below:$
$\begin{enumerate}$
$    \item  With band 7 single-pointing observation of GG Tau A, we achieved a spectral resolution of 0.1 km/s and an angular resolution of 0.8-0.9". We spatially resolved the emission from \nnhp and \dcop  while \hhdp, \CS, and \soo show no detection.$
$    \item \textbf{Comparing the predictions of a parametric disk model to our observed visibilities along with the lower excitation transition observations obtained with NOEMA by [Phuong, Chapillon and Majumdar (2018)]() and [Phuong, Dutrey and Diep (2020)](), we constrain the average temperature of \ce{N2H+} and \ce{DCO+} emitting regions to be around 12 K and 16 K respectively. This is at odds with the predictions from all the chemical models we explored and all the previous studies. Chemical models suggest \ce{DCO+} peaks at 20-25 K, where some \ce{CO} remains in the gas phase, and \ce{N2H+} peaks around 20 K, where \ce{CO} is fully frozen onto grains. On the contrary, our constraints indicate that both molecules trace regions below the CO layer  ([Phuong and Dutrey 2020]()) , much closer to the midplane than previously thought, with \ce{N2H+} being the closest.}$
$    \item \textbf{Our radial distribution of \dcop emission reveals an atypical partially overlapping double ring structure. The inner ring peaks at 210 \pm 50 au within the continuum ring region, while the more intense outer ring peaks at 375 \pm 10 au in the outer disk. We hypothesize that \ce{DCO+} forms via two different pathways in the ring and the outer disk regions depending on gas-phase CO abundance. In the ring region, \ce{DCO+} forms through isotope exchange after \ce{HCO+} arises from \ce{CO} protonation.}$
$    \item \textbf{\nnhp emission shows a ring-like radial distribution facilitated through protonation of \ce{N2} with its emission layer situated right below the \ce{DCO+} layer. The anti-correlation between \ce{N2H+} distribution with \ce{CO} distribution is clearly visible in our modelled simulation.}$
$    \item \textbf{We ran numerous chemical models varying CRI rates, C/O ratios, and stellar UV fluxes. These models suggest that the chemistry of \ce{H2D+}, \ce{N2H+}, and \ce{DCO+} in cold, dense regions is primarily influenced by gas-phase \ce{CO} abundance and the dominant midplane ion, \ce{H3+}. These regions are shielded from external UV radiation, making CRI the most crucial factor in determining \ce{CO} and \ce{H3+} abundances and, consequently, the distribution of \ce{N2H+}, \ce{DCO+}, and \ce{H2D+}. On the other hand, the chemistry of \ce{^{13}CS} and \ce{SO2} is very sensitive to the C/O ratio.}$
$     \item \textbf{After running a grid of astrochemical models to simultaneously fit the surface density constraints from the observed transitions along with the upper limits newd by the non-detections, our best-fit model, with a carbon-to-oxygen ratio of 1.0 and a sub-interstellar cosmic ray ionization rate of 10^{-18} s^{-1}, successfully reproduced the radial trends in surface density for \ce{DCO+} but slightly overestimated both the \ce{N2H+} content and the expected rotation temperatures. The CRI attenuation in the high-density GG Tau A ring can be attributed to an increased number of collisions experienced by CR particles as they gyrate towards the midplane.}$
$    \item The non-detection of \hhdp (or \textbf{at best a hint of presence at the 1-2\sigma level}) suggests that ionization is much lower in the disk-midplane due to its very high density. In the cold midplane region, \ce{H2D+} is directly dependent on the abundance of \ce{H3+}, which, in turn, is regulated by the ionization rate. Our best-fit model also highlights a low cosmic ray ionization rate (\zeta_{\text{CR}}) of 10^{-18} s^{-1}. The non-detection of \ce{H2D+} aligns with our best model, where the expected surface density is below our observationally determined 3\sigma upper limit.$
$\end{enumerate}$
$The GG Tau A disk is very complex, but it is one of the nearest (hence larger) and more massive disks found around a low-mass star system. Such properties should facilitate the detection of \hhdp. However, its modelled surface density being close to the observationally constrained 3\sigma upper limit, suggests that \textbf{the detection of \ce{H2D+} is within the ALMA capabilities, although at the cost of a significantly longer integration time.}$
$\section*{Acknowledgments}$
$This paper makes use of the following ALMA data: ADS/JAO ALMA\#2021.1.00342.S. ALMA is a partnership of ESO (representing its member states), NSF (USA) and NINS (Japan), together with NRC (Canada), MOST and ASIAA (Taiwan), and KASI (Republic of Korea), in cooperation with the Republic of Chile. The Joint ALMA Observatory is operated by ESO, AUI/NRAO and NAOJ. The National Radio Astronomy Observatory is a facility of the National Science Foundation operated under a cooperative agreement by Associated Universities, Inc. L.M. acknowledges the financial support of DAE and DST-SERB research grants (SRG/2021/002116 and MTR/2021/000864) of the Government of India. A.D. and S.G. thank the French CNRS programs PNP, PNPS, and PCMI. This$
$research was carried out in part at the Jet Propulsion Laboratory, which is operated for NASA by the California Institute of Technology. K.W. acknowledges the financial support from the NASA Emerging Worlds grant 18-EW-182-0083. Th.H. and D.S. acknowledge support from the European Research Council under the Horizon 2020 Framework Program via the ERC Advanced Grant Origins 83 24 28 (PI: Th. Henning). N.T.P. acknowledges the financial support of Vingroup Innovation Foundation (VINIF) under project code VINIF.2023.DA.057".$
$A.C. received financial support from the European Research Council (ERC) under the European Union’s Horizon 2020 research and innovation programme (ERC Starting Grant “Chemtrip”, grant agreement No 949278). \textbf{We would like to thank the anonymous referees for their constructive comments, which helped improve the manuscript.}$
$\facility{ALMA}$
$\software{\texttt{CASA}  ([McMullin, et. al 2007]()) , \texttt{keplerian\_mask}  (kepmask) , \texttt{GoFish}  (GoFish) , \texttt{VISIBLE}  (loomis2018visible) , \texttt{dynesty}  (speagle2020dynesty) , \textsc{Imager} (\url{https://imager.oasu.u-bordeaux.fr}), \texttt{DNAUTILUS}  ([Majumdar, Gratier and Ruaud 2017]()) , \texttt{numpy}  (numpy) , astropy  ([Robitaille and Tollerud 2013](), [Price-Whelan and Sip\Hocz 2018](), [Price-Whelan and Lim 2022]()) , \texttt{matplotlib}  ([ and Hunter 2007]()) }$
$\appendix$
$\section{Channel Maps, Moment Maps and Radial Profiles}$
$\label{app:ch_maps}$
$\counterwithin{figure}{section}$
$\setcounter{figure}{0}$
$The generated channel maps for the detected transition, \nnhp and \dcop are shown in Figures \ref{fig:ch_maps_n2hp} and \ref{fig:ch_maps_dcop}, respectively. Figure \ref{fig:m0_without_kepmask} shows integrated intensity maps between the velocity range 5.0 km/s and 8.0 km/s for all the targeted transitions. Sigma clipping and masking are not performed while generating these integrated intensity maps. Figure \ref{fig:rad_profile_wihtout_kep} shows the radial distribution of azimuthally averaged emission for the targeted transitions and the continuum generated from integrated intensity maps shown in Figure \ref{fig:m0_without_kepmask}. We have displayed the matched filter responses for all the targeted transitions in Figure \ref{fig:matched_filter}. The methods for generating all these figures are explained in Section \ref{subsec:obs_res}.$
$\begin{figure}[ht]$
$    \centering$
$    \includegraphics[width=\textwidth]{channel_maps_65_n2hp.pdf}$
$    \caption{Channel maps of \nnhp imaged with \texttt{tclean} task in \texttt{CASA} as explained Section \ref{sec:obs}. The channel velocities (LSRK) are displayed in the top right corner of each channel in km/s. The solid contours are [3,5,7,9,...]\times \sigma levels, where \sigma = 14.6mJy/beam. The ellipse at the lower left corner indicates the beam size of 0.93" \times 0.77" with PA 68.76^\circ. The colorbar is stretched non-linearly to showcase the line emissions more prominently.}$
$    \label{fig:ch_maps_n2hp}$
$\end{figure}$
$\begin{figure}$
$    \centering$
$    \includegraphics[width=\textwidth]{channel_maps_65_dcop.pdf}$
$    \caption{Channel maps of \dcop imaged with \texttt{tclean} task in \texttt{CASA} as explained Section \ref{sec:obs}. The channel velocities (LSRK) are displayed in the top right corner of each channel in km/s. The solid contours are [3,5,7,9,...]\times \sigma levels, where \sigma = 5.6mJy/beam. The ellipse at the lower left corner indicates the beam size of 0.91" \times 0.73" with PA 99.20^\circ. The colorbar is stretched non-linearly to showcase the line emissions more prominently.}$
$    \label{fig:ch_maps_dcop}$
$\end{figure}$
$\begin{figure}$
$    \centering$
$    \includegraphics[width = \textwidth]{M0_maps_23.pdf}$
$    \caption{Integrated Intensity Maps of the targeted transitions generated without sigma clipping and masking. The overplotted elliptical ring contours in the white are at 193 au and 285 au to indicate the continuum ring region where 90\% of the circumtertiary emission arises  ([Guilloteau, Dutrey and Simon 1999]()) . The subplot at the bottom left corner is the continuum image at 358 GHz.}$
$    \label{fig:m0_without_kepmask}$
$\end{figure}$
$\begin{figure}$
$    \centering$
$\includegraphics[width=\textwidth]{Integrated_intensity_radial_without_kep.pdf}$
$    \caption{Azimuthally averaged velocity integrated radial profiles of the detected lines generated from the integrated intensity maps without any keplerian masking. The shaded region in the above figure corresponds to 1\sigma rms. The grey horizontal line at the top right denotes the beam major axis. The dashed vertical lines are to designate the location of the ring around GG Tau A, spanning from 193 au to 285 au. The subplot at the bottom right corner is the radial distribution of the azimuthally averaged continuum emission at 358 GHz.}$
$    \label{fig:rad_profile_wihtout_kep}$
$\end{figure}$
$\begin{figure}$
$    \centering$
$    \includegraphics[width=\textwidth]{filter_response.pdf}$
$    \caption{Matched filter responses to the emission for the targeted transitions, written at the upper left corner. The vertical grey dashed line is plotted at source velocity v_{\text{LSRK}} = 6.4 km/s for each subplot. Only \nnhp and \dcop show clear detection; rest of them do not.}$
$    \label{fig:matched_filter}$
$\end{figure}$
$\section{IMAGER and DiskFit analyis} \label{app:imager}$
$\counterwithin{figure}{section}$
$\setcounter{figure}{0}$
$Figure \ref{fig:kepler-n2h} shows the$
$Keplerian deprojection produced by \textbf{the \textsc{Imager} command} \texttt{KEPLER}. Images used for this purpose \textbf{were produced with \textsc{Imager}  and}$
$have an angular resolution of 0.66 \times 0.53"$
$at PA 83^\circ. Spectral resolution prior to$
$\textbf{Keplerian velocity correction was about 0.1 km s^{-1}. This correction requires resampling of the spectra on a different grid, which results in correlated channels: the correlation factor is estimated from the noise spectrum autocorrelation.}$
$The best fit radial profile derived from$
$\textsc{DiskFit} is always slightly above the measured one because the deconvolution with$
$limited signal to noise cannot recover the full line brightness, while the uv plane fitting method is not biased in this respect.$
$\begin{figure}$
$    \centering$
$    \includegraphics[width=8.9cm]{kep-n2h+43.pdf}\hspace{0.1cm}$
$    \includegraphics[width=8.9cm]{kep-dco+54.pdf}$
$\caption{Kepler plots generated for the observed transitions of \nnhp (left) and \dcop (right). For each molecule, the map is the brightness temperature (in K) as a function of velocity and radius$
$("teardrop" plot). The left panel shows the peak brightness radial profile, with error bars indicated, \textbf{and the red profile being the one derived from the best fit DiskFit model. The top panel shows the integrated spectrum over the range indicated by the blue line in the radial profile, with the best-fit Gaussian spectrum superimposed in red. The peak (S, in Jy) and integrated$
$(\int S_\nu dV, in Jy.km/s) flux densities, central velocity (V in km/s), and FWHM line width (\Delta V in km/s) obtained from a Gaussian fit to the integrated spectrum} are given in the upper left corner of each panel.}$
$\label{fig:kepler-n2h}$
$\end{figure}$
$\section{Simple LTE Approach}$
$\label{sec:rotational_diagram}$
$We have undertaken a simpler approach to calculate disk-averaged surface densities where \textbf{we assume local thermodynamic equilibrium (LTE) considering the disk densities are relatively high compared to critical densities for the studied transitions.} This method requires prior knowledge of rotational temperature for a single transition.\par$
$For optically thin emission, the surface density of molecules in the upper state of a specific transition is expressed as,$
$\begin{align}$
$N_u^{thin} &=$
$\frac{4\pi}{ h c A_{ul}} \frac{(\int S_\nu dv)}{\Omega} \ &=$
$\frac{8\pi k \nu^2 }{ h c^3 {A_{ul}}  } \left(\int T_b dv\right)$
$\end{align}$
$\noindent$
$where \int S_\nu dv is the velocity integrated flux density, A_{ul} is the Einstein coefficient, \Omega denotes the solid angle subtended by the emission area, and \textit{h} and \textit{c} stand for the Planck constant and the speed of light in a vacuum, respectively  (goldsmith1999population) . In the alternate form,$
$\nu is the line frequency and T_b \textbf{the line brightness temperature.}$
$The total surface density can be derived from the upper state surface density, assuming a Boltzmann distribution$
$\begin{align}$
$    N_T^{thin} = N_u^{thin}  \frac{Q(T_{rot})}{g_u} exp \left(\frac{E_u}{T_{rot}}\right) \label{eq:Nt_thin}$
$\end{align}$
$\noindent$
$where g_u is the degeneracy, E_u is the energy of the upper energy level in K, and \textit{Q(T_{rot})} is the partition function at the rotation  temperature T_{rot}, that can be computed by summation$
$over the energy levels or by interpolating tabulated Q values for discrete T_{ex} values found in the Cologne Database for Molecular Spectroscopy (CDMS)  (endres2016cologne) .\par$
$If the transition is optically thick, the above value gives$
$a strict lower limit to the observed surface density.$
$Opacity corrections are possible in a purely homogeneous medium with only turbulent or thermal line broadening  (goldsmith1999population) :$
$\begin{align}$
$    N_T = N_T^{thin} \frac{\tau}{1- e^{-\tau}} \label{eq:tau_correction}$
$\end{align}$
$where we can self-consistently calculate the optical depth, \tau assuming a Gaussian-like line profile \sim 1/\Delta v at the line center \citep[][]{goldsmith1999population}, \Delta v = 0.25 km/s being the intrinsic full-width half maximum line width constrained from \textsc{DiskFit} analysis:$
$\begin{align}$
$\tau  = \frac{A_{ul}c^3}{8\pi \nu^3 \Delta v} N_u$
$    \left[ \exp\left(\frac{h \nu}{k T_{rot}} \right) - 1 \right]$
$    \label{eq:tau}$
$\end{align}$
$Unfortunately, in our case, the above-mentioned correction does not hold while calculating the radially resolved surface density, even if we apply this to radially dependent emission. Because of the Keplerian shear, there are significant regions in our images where the line of sight opacity varies both as a function of azimuth and radius, with both directions being insufficiently well resolved at our angular resolution (about 100 au). Accordingly, the azimuthally averaged brightness does not originate from a unique opacity below about 400 au.\par$
$\textbf{However, the effect is expected to be small.$
$ The peak brightness of N_2H^+ (see Figure \ref{fig:kepler-n2h}) is 2 K, while the brightness temperature of an optically thick line would be J_\nu(T_{rot}) = 5.3 K for T_{rot} = 12 K at the J=4-3 line frequency. This indicates a peak line opacity of about 0.47, leading to a \sim 22\% opacity correction at most. For DCO^+, whose emission is about 3 times fainter, the correction is even smaller, as for the (undetected) emission from H_2D^+ and other molecules.} We thus expect the constraint given by the optically thin approximation to be a reasonable estimate for the mean surface density.\par$
$\textbf{We explore the likely ranges of disk-averaged surface density using nested sampling while assuming log-normal priors for both the molecules in the range 10^5 < N_{\text{T}} < 10^{18} cm^{-2} and fixing the rotational temperature to the values constrained in \textsc{DiskFit} analysis.} The Python package utilized to sample the posterior distribution is \texttt{dynesty}  (speagle2020dynesty) . We consider the median of the posterior as the best-fit value, while the 16th and 84th percentile values account for the 1 \sigma uncertainties.$
$Posterior distributions for these averaged surface density derivations are shown in Figure \ref{fig:posterior}.$
$\textbf{The rotation temperatures for N_2H^+ and DCO^+ were set at 12 K and 16 K, respectively from the \textsc{DiskFit} results. A value of 15 K$
$was used for undetected molecules.}$
$\begin{figure}$
$    \centering$
$    \includegraphics[width=\textwidth]{posterior.pdf}$
$    \caption{Posterior Distribution for disk averaged surface density calculations using the simple LTE approach. The left panel is for molecules \ce{N2H+}, and the right one is for \ce{DCO+}. The grey dashed lines represent 16, 50, and 84th percentile of the distribution. The median value with 1 \sigma error bars (68\% confidence interval) are reported at the top left.}$
$    \label{fig:posterior}$
$\end{figure}$
$\section{Grids of Astrochemical Models and Parameter Space Exploration} \label{app:chemical_model}$
$\counterwithin{figure}{section}$
$\setcounter{figure}{0}$
$We ran a large number of astrochemical models with various combinations of parameters. The grids of models are summarized in Table \ref{tab:dnautilus_parameters}. Note that we have not implemented any grain growth in our models due to poor constraints on larger grain size.$
$\begin{deluxetable*}{cccccc}[b]$
$\tablecaption{Parameter Space Explored with \text{DNAUTILUS}}$
$\label{tab:dnautilus_parameters}$
$\tablewidth{\textwidth}$
$\tablehead{$
$  \colhead{Initial Abundances} & \colhead{Settling} & \colhead{Grain Growth} & \colhead{f_{\text{UV, 214 \text{ au}}}^a(\chi_0)} & \colhead{C/O^b} & \colhead{\zeta_{\text{CR}}^c (s^{-1})}$
$}$
$\startdata$
$Atomic^d & no & no & 375, 1500, 3000 & 0.5, 0.7, 0.9, 1.2 & 10^{-16}, 10^{-17}, 10^{-18}, 10^{-19}, 10^{-20} \ 10^6 yr Molecular Cloud^e & no & no & 375, 1500, 3000 & 0.5, 0.7, 0.9, 1.2 & 10^{-16}, 10^{-17}, 10^{-18}, 10^{-19}, 10^{-20} \ Atomic^d & yes & no & 375, 1500, 3000 & 0.5, 0.7, 0.9, 1.2 & 10^{-16}, 10^{-17}, 10^{-18}, 10^{-19}, 10^{-20} \ 10^6 yr Molecular Cloud^e & yes & no & 375, 1500, 3000 & 0.5, 0.7, 0.9, 1.2 & 10^{-16}, 10^{-17}, 10^{-18}, 10^{-19}, 10^{-20} \ \enddata$
$\tablecomments{$
$a) f_{UV} is the stellar UV flux at reference radius, 214 au in the units of Draine interstellar UV field, \chi_0$
$b) Carbon to Oxygen (C/O) ratios have been considered around the standard value of 0.7 to account for "low depletion" and "high depletion" of oxygen  ([Reboussin, et. al 2015]()) .$
$c) Different cosmic ray ionization rates \zeta_{\text{CR}} have been explored to account for how they impact our chemistry. d) In the reset scenario, we have taken our initial abundances in atomic forms along with \ce{H2} and \ce{HD} and run our astrochemical model for 10^6 years. e) We have also considered the inheritance scenario as another possibility, where we have run a cloud model with atomic initial abundances, and the final abundances after 10^6 years are used as initial abundance in the disk model. The disk model was then run for 10^6 years.$
$}$
$\end{deluxetable*}$
$\clearpage$
$\bibliography{main} $
$\bibliographystyle{aasjournal}$
$\end{document}}\end{document}}$</div>



<div id="title">

# $\large{Chemistry in the GG Tau A Disk: Constraints from $H_2$D$^+$, $N_2$H$^+$, and DCO$^+$ High Angular Resolution ALMA Observations}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.07238-b31b1b.svg)](https://arxiv.org/abs/2407.07238)<mark>Appeared on: 2024-07-11</mark> -  _Resubmitted to ApJ. Thanks to the two anonymous referees for their constructive comments_

</div>
<div id="authors">

P. Kashyap, et al. -- incl., <mark>D. Semenov</mark>, <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** Resolved molecular line observations are essential for gaining insight into the physical and chemical structure of protoplanetary disks, particularly in cold, dense regions where planets form and acquire compositions. However, tracing these regions is challenging because most molecules freeze onto grain surfaces and are not observable in the gas phase. We investigated cold molecular chemistry in the triple stellar T Tauri disk GG Tau A, which harbours a massive gas and dust ring and an outer disk, using ALMA Band 7 observations. We present high angular resolution maps of $\ce{N2H+}$ and $\ce{DCO+}$ emission, with upper limits reported for $\ce{H2D+}$ , $^{13}$ CS, and $\ce{SO2}$ . The radial intensity profile of $\ce{N2H+}$ shows most emission near the ring outer edge, while $\ce{DCO+}$ exhibits double peaks, one near the ring inner edge and the other in the outer disk. With complementary observations of lower-lying transitions, we constrained the molecular surface densities and rotation temperatures. We compared the derived quantities with model predictions across different cosmic ray ionization (CRI) rates, carbon-to-oxygen (C/O) ratios, and stellar UV fluxes. **Cold molecular chemistry, affecting \ce{N2H+**, \ce{DCO+}, and \ce{H2D+} abundances, is most sensitive to CRI rates, while stellar UV flux and C/O ratios have minimal impact on these three ions. Our best model requires a low cosmic ray ionization rate of $10^{-18}$ s$^{-1}$. However, it fails to match the low temperatures derived from \ce{N2H+} and \ce{DCO+}, 12 to 16 K, which are much lower than the CO freezing temperature.}

</div>

<div id="div_fig1">

<img src="tmp_2407.07238/./Integrated_intensity_radial_14th.png" alt="Fig9" width="100%"/>

**Figure 9. -** Azimuthally averaged velocity integrated radial profiles of the detected lines generated from the integrated intensity maps shown in Figure \ref{fig:mo_maps}. The shaded region in the above figure corresponds to 1$\sigma$. The dashed vertical lines are to designate the location of the ring around GG Tau A, spanning from 193 au to 285 au. The grey horizontal line at the top right corner represents the beam major axis. (*fig:radial_intensity*)

</div>
<div id="div_fig2">

<img src="tmp_2407.07238/./kep-n2h+43.png" alt="Fig6.1" width="50%"/><img src="tmp_2407.07238/./kep-dco+54.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** Kepler plots generated for the observed transitions of $\nnhp$(left) and \dcop(right). For each molecule, the map is the brightness temperature (in K) as a function of velocity and radius
("teardrop" plot). The left panel shows the peak brightness radial profile, with error bars indicated, $\textbf${and the red profile being the one derived from the best fit DiskFit model. The top panel shows the integrated spectrum over the range indicated by the blue line in the radial profile, with the best-fit Gaussian spectrum superimposed in red. The peak ($S$, in Jy) and integrated
($\int S_\nu dV$, in Jy.km/s) flux densities, central velocity ($V$ in km/s), and FWHM line width ($\Delta V$ in km/s) obtained from a Gaussian fit to the integrated spectrum} are given in the upper left corner of each panel. (*fig:kepler-n2h*)

</div>
<div id="div_fig3">

<img src="tmp_2407.07238/./int_spetra.png" alt="Fig10" width="100%"/>

**Figure 10. -** Keplerian deprojected and stacked spectra of the detected lines $\nnhp$ and \dcop. The shaded region is the region over which we have done a Gaussian fitting and calculated integrated flux densities. The stacked spectra are peaking at the systemic velocity, $v_{\text{LSRK}}$ of 6.4 km/s, confirming the detections. The location of the systemic velocity is shown by the vertical dashed lines. Integrated flux densities and matched filter responses are tabulated in Table \ref{tab:obs_lines}. (*fig:int_spectra*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.07238"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\SEM}[1]{{\color{cyan} #1}}$
$\newcommand\ra{#1#2#3}$
$\newcommand\dec{#1#2#3}$
$\newcommand{\Msol}{\rm{M_{\odot}}}$
$\newcommand{\nod}{\nodata}$
$\newcommand{\swift}{{\it Swift}}$
$\newcommand{\chandra}{{\it Chandra}}$
$\newcommand{\sn}{AT 2020xnd}$
$\newcommand{\cow}{AT 2018cow}$
$\newcommand{\koala}{ZTF18 abvkwla}$
$\newcommand{\css}{CSS 161010}$</div>



<div id="title">

# Constraints on Relativistic Jets from the Fast X-ray Transient 210423 using Prompt Radio Follow-up Observations

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.07257-b31b1b.svg)](https://arxiv.org/abs/2407.07257)<mark>Appeared on: 2024-07-11</mark> -  _14 pages, 6 figures_

</div>
<div id="authors">

D. Ibrahimzade, et al. -- incl., <mark>K. Paterson</mark>

</div>
<div id="abstract">

**Abstract:** Fast X-ray Transients (FXTs) are a new observational class of phenomena with no clear physical origin. This is at least partially a consequence of limited multi-wavelength follow up of this class of transients in real time. Here we present deep optical ( $g-$ and $i-$ band) photometry with Keck, and prompt radio observations with the VLA of FXT 210423 obtained at $\delta t \approx 14-36$ days since the X-ray trigger. We use these multi-band observations, combined with publicly available data sets, to constrain the presence and physical properties of on-axis and off-axis relativistic jets such as those that can be launched by neutron-star mergers and tidal disruption events, which are among the proposed theoretical scenarios of FXTs. Considering a wide range of possible redshifts $z\le3.5$ , circumstellar medium (CSM) density $n=10^{-6}-10^{-1} \rm{cm^{-3}}$ , isotropic-equivalent jet kinetic energy $E_{k,iso}=10^{48}-10^{55} \rm{erg}$ , we find that we can rule out wide jets with opening angle $\theta_{j}=15◦ee$ viewed within $10◦ee$ off-axis. For more collimated jets ( $\theta_{j}=3◦ee$ ) we can only rule out on-axis ( $\theta_{j}=0◦ee$ ) orientations. This study highlights the constraining power of prompt multi-wavelength observations of FXTs discovered in real time by current (e.g., Einstein Probe) and future facilities.

</div>

<div id="div_fig1">

<img src="tmp_2407.07257/./figures/Grid_theta_j_15_deg_updated_july24.png" alt="Fig5" width="100%"/>

**Figure 5. -** Grid representation of simulations that violate observations for a jet with $\theta_j =  15◦ee$. Violations occur when either the radio, optical, or X-ray value of an observation exceeds that of the simulation at the corresponding observation time. Colored rings indicate a violation in a particular band (green: radio, purple: optical, yellow: X-ray). The size of the ring indicates  the observation angle of the simulation where the violation occurred with the innermost ring representing a violation of 0$◦ee$ and each proceeding concentric ring representing an increase of 10$◦ee$.
A set of 9 concentric rings, in any colour, consequently indicates that any jet with these properties are ruled out. Columns and rows of the the outer grid indicate the energy and distance of the simulation. Columns and rows of each inner box indicate the fraction of the post shock energy transferred into magnetic fields ($\epsilon_B$) and the density of the surrounding medium ($n$) in the simulation. The violation results are overlaid in the order radio (top), optical (middle), and X-ray (last), meaning the appearance of a radio or optical violation may cover an X-ray violation. The ordering has been chosen to best represent the overall shape of the violations. Any crossing on the grid with no ring indicates no constraints could be placed on simulations with the given parameters.
 (*figure:Grid_15_deg*)

</div>
<div id="div_fig2">

<img src="tmp_2407.07257/./figures/Light_Curve_with_HR_july24.png" alt="Fig1" width="100%"/>

**Figure 1. -** Unabsorbed 0.5-7 keV flux (*main panel*) and hardness ratio (*upper panel*) evolution of FXT 210423.
The soft band (S) is defined between 0.5-2 keV, and the hard band (H) between  2-7 keV. No spectral evolution is apparent. The X-ray light-curve can be fit with a broken power law (red solid line)  with $F_{X} \propto t^{-0.2 \pm 0.2}$ at $\delta t<4.1 \rm{ks}$ steepening to $F_{X} \propto t^{-4.1 \pm 1.4}$
at later times. (*figure:LC*)

</div>
<div id="div_fig3">

<img src="tmp_2407.07257/./figures/Optical_hosts.png" alt="Fig4" width="100%"/>

**Figure 4. -** Keck $g$-band image of the field of FXT 210423 acquired on 2021 May 11 ($\delta t=$18.03 d since discovery). Red region: 0.7$\arcsec$ radius region at the location of the X-ray source from the CXO discovery images as determined with \texttt{wavdetect}. Cyan region: 0.7$\arcsec$ radius region at the location of the extended source that we have identified in our Keck $g$-band images. We also mark the three extended sources ("cX", "cW" and "cNE") identified as potential host galaxies by  eappachen2023fast. Our Keck source is coincident with "cX" and we consider "cX" the most likely host galaxy for this FXT.  North is up and East is right.
 (*figure:Optical_hosts*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.07257"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# A photo-$z$ cautionary tale: Redshift confirmation of COSBO-7 at $z=2.625$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.07585-b31b1b.svg)](https://arxiv.org/abs/2407.07585)<mark>Appeared on: 2024-07-11</mark> -  _Submitted to A&A Letter_

</div>
<div id="authors">

S. Jin, et al. -- incl., <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** Photometric redshifts are widely used in studies of dusty star-forming galaxies (DSFGs), but catastrophic photo- $z$ failure can undermine all redshift-dependent results. Here we report the spectroscopic redshift confirmation of COSBO-7, a strongly lensed DSFG in the COSMOS-PRIMER field.Recently, using 10 bands of JWST NIRCam and MIRI imaging data on COSBO-7, Ling et al. (2024) reported a photometric redshift solution of $z\gtrsim7.0$ , favored by four independent spectral energy distribution (SED) fitting codes, and therefore providing an appealing candidate of the most distant massive DSFG. This photo- $z$ solution was also supported by a single line detection in ALMA Band 3 consistent with CO(7-6) at $z=7.46$ .However, our new ALMA observations robustly detect two lines in Band 6 identified as CO(7-6) and [ CI ] (2-1) at $z_{\rm spec}=2.625$ , and thus the Band 3 line as CO(3-2). The three robust line detections decidedly place COSBO-7  at $z=2.625$ , refuting the photo- $z$ solution.We derive physical parameters by fitting NIR-to-mm photometry and lens modeling, revealing that COSBO-7 is a main-sequence galaxy.We examine possible reasons for this photo- $z$ failure and attribute it to (1) the likely underestimation of photometric uncertainty at 0.9 $\mu$ m, and (2) the lack of photometry at wavelengths beyond 20 $\mu$ m.Notably, we recover a bona-fide $z_{\rm phot}\sim 2.3$ by including the existing MIPS $24 \mu$ m photometry, demonstrating the critical importance of mid-infrared data for photo- $z$ robustness.This work highlights a common challenge in modeling SEDs of DSFGs, cautioning against the reliability of photometric redshifts, as well as pseudo-spectroscopic redshifts based on single line detection.

</div>

<div id="div_fig1">

<img src="tmp_2407.07585/./fig3.png" alt="Fig5" width="100%"/>

**Figure 5. -** NIR-to-MIR SED of COSBO-7 fitted with \texttt{Bagpipes}. _ Left:_ The fitting without F090W upper limit and $24\mu$m photometry, with the PDF($z$) shown in sub-panel. We also show the SEDs and $\chi^2$ for both $z=2.625$ and $7.458$ cases. _ Right:_ The best-fit with MIPS $24\mu$m photometry. We present the PDF($z$) with $z_{\rm spec}$ in sub-panel.
 (*fig:NIR-sed*)

</div>
<div id="div_fig2">

<img src="tmp_2407.07585/./fig1.png" alt="Fig4" width="100%"/>

**Figure 4. -** JWST images and ALMA spectra of COSBO-7. _ Top:_ We show the NIRCam color image (Blue: F090W+F115W+F150W; Green: F200W+F277W; Red: F356W+F410M+F444W), and MIRI images overlaid with contours of CO and [CI] emission. Contours are shown in 4, 6, 8, 10$\sigma$ levels. The beams are shown as dashed ellipses. _ Bottom:_ In the left and middle panels, we show the CO(3-2) and CO(7-6)+[CI](2-1) spectra in observed frequencies. The right panel shows the continuum-subtracted spectra as a function of velocity. (*fig-spec*)

</div>
<div id="div_fig3">

<img src="tmp_2407.07585/./COSBO7_FIR_SED.png" alt="Fig1" width="100%"/>

**Figure 1. -** Panchromatic SED of COSBO-7 fitted with \texttt{STARDUST} ([Kokorev, Magdis and Davidzon 2021]()) . The F090W upper limit is shown in $3\sigma$ level. Radio photometry are not included in the fitting, we extrapolated a radio component using the IR luminosity and the IR-radio relation from [Delvecchio, Daddi and Sargent (2021)](). Parameters are not corrected for lensing magnification.
 (*fig:sed*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.07585"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# From traffic jams to roadblocks: The outer regions of TW Hya with ALMA Band 8

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.07649-b31b1b.svg)](https://arxiv.org/abs/2407.07649)<mark>Appeared on: 2024-07-11</mark> -  _Accepted for publication in A&A. The associated data can be found in: this https URL_

</div>
<div id="authors">

S. Das, N. Kurtovic, <mark>M. Flock</mark>

</div>
<div id="abstract">

**Abstract:** ${Current ALMA surveys often underestimate protoplanetary disk sizes as many disks have extended low surface-brightness regions that fall below ALMA's detection limits. To effectively capture faint millimeter continuum emission in these outer regions, increased sensitivity is required.}$ In order to gain insights on the ${connection between disk structure and}$ planet formation, we aim to uncover ${continuum}$ emission in the outer regions of the disk around TW Hya. ${Additionally, we aim to investigate the evolution of this disk by studying its dust properties.}$ We present Atacama Large Millimeter Array (ALMA) observations of TW Hya at 0.65 mm with $\sim$ 0.5 arcsecond angular resolution, together with high angular resolution archival ${observations}$ at 0.87 mm, 1.3 mm, 2.1 mm and 3.1 mm. We constrain the outer disk emission ${with both image-plane retrieval, and visibility-plane modeling with non-parametric and parametric fitting tools.}$ Our results confirm emission in the outer disk regions of TW Hya (60 au $\leq$ R $\leq$ 110 au) at 0.65 mm, 0.87 mm and 1.3 mm. ${With image-plane retrieval}$ , we resolve the new continuum gap and ring, namely D79 and B86, at 0.87 mm and 1.3 mm. ${With visibility-plane modeling}$ , we also detect this substructure at 0.65 mm, and it is consistent in location with the outer ring proposed by \citealp{ilee}{at the 1$\sigma$ level} . ${Furthermore, it}$ has a high spectral index of 3.7, which may indicate dust grain sizes $\ll$ 1 mm. It may be a dust trap or a traffic jam, that has a flux density of $\sim$ 60 mJy and a mass (1.59 $M_\oplus$ ) that accounts for up to 2 \% of the dust disk at 0.65 mm. We confirm the existence of a faint ring in the outer regions of TW Hya at multiple millimeter wavelengths. ${With visibility-plane modeling, we are able to set constrains that are 3 times better than the resolution of our Band 8 observations.}$

</div>

<div id="div_fig1">

<img src="" alt="Fig5.1" width="25%"/><img src="" alt="Fig5.2" width="25%"/><img src="" alt="Fig5.3" width="25%"/><img src="" alt="Fig5.4" width="25%"/>

**Figure 5. -** Vibrational stability equation of state
               $S_{\mathrm{vib}}(\lg e, \lg \rho)$.
               $>0$ means vibrational stability.
              Vibrational stability equation of state
               $S_{\mathrm{vib}}(\lg e, \lg \rho)$.
               $>0$ means vibrational stability.
              Nonlinear Model ResultsNonlinear Model ResultsSpectral types and photometry for stars in the
  region.Spectral types and photometry for stars in the
  region.List of nearby SNe used in this work.Summary for ISOCAM sources with mid-IR excess
(YSO candidates).Summary for ISOCAM sources with mid-IR excess
(YSO candidates). Sample stars with absolute magnitudecontinued. Sample stars with absolute magnitudecontinued.Shown in greyscale is a...Plotted above...Complexes characterisation.Line data and abundances ...Continued. (*FigVibStab*)

</div>
<div id="div_fig2">

<img src="tmp_2407.07649/./UV_plot.png" alt="Fig7.1" width="50%"/><img src="tmp_2407.07649/./MCMC_Intensity_Profile.png" alt="Fig7.2" width="50%"/>

**Figure 7. -** {Results of parametric modeling}: _(Left)_ The re-centered and deprojected visibilities of the 0.65 mm observation binned to 10 k$\lambda$, where the top row shows the real part and the bottom row shows the imaginary part. The visibilities of the best fit model is shown in red. _(Right)_ Intensity profile of the TW Hya protoplanetary disk at 0.65 mm made with 5000 random posteriors of our MCMC sample. The median intensity profile is shown with a black curve, whereas the 1$\sigma$ and 3$\sigma$ uncertainties are shown with dark and light red shaded regions respectively. The local minima and maxima representing a dark gap and a bright ring at 86 au and 96 au are shown with dotted and solid lines respectively. The disk size estimates are shown as $R_{68}$, $R_{95}$ and $R_{99}$ in the form of gray columns. (*fig:mcmc_profile*)

</div>
<div id="div_fig3">

<img src="" alt="Fig12.1" width="33%"/><img src="" alt="Fig12.2" width="33%"/><img src="" alt="Fig12.3" width="33%"/>

**Figure 12. -** Vibrational stability equation of state
               $S_{\mathrm{vib}}(\lg e, \lg \rho)$.
               $>0$ means vibrational stability.
              Nonlinear Model ResultsNonlinear Model ResultsSpectral types and photometry for stars in the
  region.Spectral types and photometry for stars in the
  region.List of nearby SNe used in this work.Summary for ISOCAM sources with mid-IR excess
(YSO candidates).Summary for ISOCAM sources with mid-IR excess
(YSO candidates). Sample stars with absolute magnitudecontinued. Sample stars with absolute magnitudecontinued.Shown in greyscale is a...Plotted above...Complexes characterisation.Line data and abundances ...Continued. (*FigVibStab*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.07649"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand{\myorcidid}{0000-0001-9282-9462}$
$\newcommand{\rsun}{R_{\odot}}$
$\newcommand{\teff}{T_{\rm eff}}$
$\newcommand{\logg}{\ensuremath{\log g}}$
$\newcommand{\Msun}{M_{\odot}}$
$\newcommand{\logl}{log(L/L_{\odot})}$
$\newcommand{\Mjup}{M_{\text{Jup}}}$
$\newcommand{\Rjup}{R_{\text{Jup}}}$
$\newcommand$
$\newcommand{\RGM}[1]{\textcolor{cyan}{#1}}$
$\newcommand{\twelveCO}{\textsuperscript{12}CO}$
$\newcommand{\thirteenCO}{\textsuperscript{13}CO}$
$\newcommand{\CeighteenO}{C\textsuperscript{18}O}$
$\newcommand{\eighteenO}{\textsuperscript{18}O}$
$\newcommand{\seventeenO}{\textsuperscript{17}O}$
$\newcommand{\isotope}[2]{\textsuperscript{#2}#1}$
$\newcommand{\ratio}[3]{\textsuperscript{#2}#1/\textsuperscript{#3}#1}$
$\newcommand{\Htwo}{H_2}$
$\newcommand{\pRT}{\texttt{petitRADTRANS}}$
$\newcommand{\micron}{\mum}$
$\newcommand{\vsini}{v\sin{i} }$
$\newcommand{\Teff}{T_{\text{eff}}}$
$\newcommand{\evidence}{\mathcal{Z} }$
$\newcommand{\Cratio}{\textsuperscript{12}C/\textsuperscript{13}C}$
$\newcommand{\Cratiosolar}{(\textsuperscript{12}C/\textsuperscript{13}C)_{\odot}}$
$\newcommand{\Cratioism}{(\textsuperscript{12}C/\textsuperscript{13}C)_{\rm ISM}}$
$\newcommand{\Oratiosolar}{(\textsuperscript{16}O/\textsuperscript{18}O)_{\odot}}$
$\newcommand{\Oratioism}{(\textsuperscript{16}O/\textsuperscript{18}O)_{\rm ISM}}$
$\newcommand{\water}{H_2O }$
$\newcommand{\CRIRES}{CRIRES^+}$
$\newcommand{\sixteenOwater}{H\textsubscript{2}\textsuperscript{16}O}$
$\newcommand{\eighteenOwater}{H\textsubscript{2}\textsuperscript{18}O}$
$\newcommand{\limbdarkening}{\varepsilon_{\rm limb}}$
$\newcommand{\Nphi}{N_{\phi}}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\kms}{\ifmmode{\rm km\th s^{-1}}\else km\th s^{-1}\fi}$
$\newcommand{\th}{\thinspace}$
$\newcommand\hyper{@linkstart##1##2 $
$    }$
$\newcommand\hyper{@linkstart##1##2 $
$    }$</div>



<div id="title">

# The $^{12}$C/$^{13}$C ratios of three young brown dwarfs with CRIRES$^+$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.07678-b31b1b.svg)](https://arxiv.org/abs/2407.07678)<mark>Appeared on: 2024-07-11</mark> -  _Accepted for publication in A&A_

</div>
<div id="authors">

D. G. Picos, et al. -- incl., <mark>P. Mollière</mark>

</div>
<div id="abstract">

**Abstract:** Young brown dwarfs exhibit atmospheric characteristics similar to those of super-Jupiters, providing a unique opportunity to study planetary atmospheres. Atmospheric retrievals of high-resolution spectra reveal detailed properties of these objects, with elemental and isotopic ratios offering insights into their formation history. The ESO SupJup Survey, utilising CRIRES $^+$ on the Very Large Telescope, aims to assess the role of $\Cratio$ as a formation tracer. We present observations of three young brown dwarfs: 2MASS J12003792-7845082, TWA 28, and 2MASS J08561384-1342242. Our goal is to constrain their chemical compositions, thermal profiles, surface gravities, spin rotations, and $\Cratio$ . We conducted atmospheric retrievals of CRIRES $^+$ K-band spectra, coupling the radiative transfer code \texttt{petitRADTRANS} with the Bayesian inference algorithm \texttt{MultiNest} . The retrievals provide a detailed characterisation of the atmospheres of the three objects. We report the volume mixing ratios of the main molecular and atomic species: $\sixteenOwater$ , $\twelveCO$ , HF, Na, Ca, and Ti, including the novel detection of hydrogen fluoride (HF) in the atmosphere of a brown dwarf. We determine $\Cratio$ values of $81^{+28}_{-19}$ and $79^{+20}_{-14}$ in the atmospheres of TWA 28 and J0856, respectively, with strong significance ( $>3\sigma$ ). We also report tentative evidence ( $\sim 2\sigma$ ) of $\thirteenCO$ in J1200, at $\Cratio$ = $114^{+69}_{-33}$ . Additionally, we detect $\eighteenOwater$ at moderate significance in J0856 (3.3 $\sigma$ ) and TWA 28 (2.1 $\sigma$ ). The retrieved thermal profiles are consistent with hot atmospheres ( $2300-2600$ K) with low surface gravities and slow spins, as expected for young objects. The measured carbon isotope ratios are consistent among the three objects and show no significant deviation from that of the local interstellar medium, suggesting a fragmentation-based formation mechanism similar to star formation. The tentative detection of $\eighteenOwater$ in two objects of our sample highlights the potential of high-resolution spectroscopy to probe additional isotope ratios, such as $\ratio{O}{16}{18}$ , in the atmospheres of brown dwarfs and super-Jupiters.

</div>

<div id="div_fig1">

<img src="tmp_2407.07678/./fig5_logg_FeH.png" alt="Fig6" width="100%"/>

**Figure 6. -** Posterior distributions of the surface gravity and carbon-to-oxygen ratio relative to the solar value for each object. The contours of the 2D histogram represent the 1, 2, and 3$\sigma$ confidence intervals, with dashed lines indicating the median values. The Pearson correlation coefficient is shown in the top-right corner, where a value of $r=0$ indicates no correlation and $r=1$ indicates a perfect positive correlation. (*fig:bestfit_logg_FeH*)

</div>
<div id="div_fig2">

<img src="tmp_2407.07678/./fig4_bestfit_PT.png" alt="Fig17.1" width="50%"/><img src="tmp_2407.07678/./fig6_chemistry.png" alt="Fig17.2" width="50%"/>

**Figure 17. -** (a) Best-fit pressure-temperature profiles. The shaded regions indicate the 1-, 2-, and  3-sigma regions. On the right axis we overplot the integrated contribution function. (b) Posterior distributions for C/O (top) and $\Cratio$(bottom). The values for the solar C/O $=0.59\pm0.08$\citep{asplundChemicalMakeupSun2021}, and the isotope ratios for the Solar System, $89.3\pm0.2$\citep{meijaIsotopicCompositionsElements2016}, and the ISM, $\Cratio$ism = $68 \pm 16$\citep{milam1213Isotope2005}, are marked. (*fig:bestfit_PT*)

</div>
<div id="div_fig3">

<img src="tmp_2407.07678/./fig_opacities_TWA28_T2400_additional_species.png" alt="Fig3" width="100%"/>

**Figure 3. -** Opacity sources included in the forward modelling. The opacities are shown for a temperature of 2400 K and at the retrieved mixing ratios of TWA 28 (see \Cref{tab:retrieval_results}). (*fig:opacity_sources*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.07678"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

128  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

18  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

9  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
